In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import ascii
from scipy.optimize import curve_fit
from lmfit import Model, minimize, Parameters, report_fit
from lmfit.models import GaussianModel, PolynomialModel
import astropy.units as u
import pandas as pd
import glob
import pandas as pd
import csv
import celerite

In [2]:
def Gauss(x, h1, c1, w1):
    G = h1*np.exp(-(x-c1)**2/(2*w1**2))
    return G

def doubleGauss(x, h1, c1, w1, h2, c2, w2):
    G = h1*np.exp(-(x-c1)**2/(2*w1**2)) + h2*np.exp(-(x-c2)**2/(2*w2**2))
    return G

def tripleGauss(x, h1, c1,w1, h2, c2, w2, h3, c3, w3):
    G = h1*np.exp(-(x-c1)**2/(2*w1**2)) + h2*np.exp(-(x-c2)**2/(2*w2**2)) + h3*np.exp(-(x-c3)**2/(2*w3**2))
    return G


In [3]:
'''def fit_coord(l, b, n_HI = 1, n_65 = 1, n_67 = 1, n_CO = 1, HI_p0 = [1, 0, 1], OH65_p0 = [0.03, 0, 4], OH67_p0 = [0.03, 0, 4], CO_p0 = [1, 0, 1], HI_bounds = (-np.inf, np.inf), OH65_bounds = (-np.inf, np.inf), OH67_bounds = (-np.inf, np.inf), CO_bounds = (-np.inf, np.inf), Cloud = []):
    
    # Setup a pandas dataframe to save all the variables we'd like
    
    '''Purpose:
    '''This function fits multiple gaussians to HI, CO and HI data and returns the parameters...
    
    Parameters:
    l: galactic latitude, int
    b: galactic lontitude, int
    HIdata: np.array of HI data in Tb, vel
    OHdata: np.array of OH data in Tb, vel
    ...

    Returns a pandas dataframe'''
    '''
    #OH67 = ascii.read('C:/Users/seven/OneDrive/Documents/DustSurvey/OH' + str(l) + 'p' + str(b) + '_67basesub.txt')
    #OH65 = ascii.read('C:/Users/seven/OneDrive/Documents/DustSurvey/OH' + str(l) + 'p' + str(b) + '_65basesub.txt')
    #HI = ascii.read('C:/Users/seven/OneDrive/Documents/DustSurvey/HI' + str(l) + 'p' + str(b) + '.txt', data_start = 13, data_end = 167)
    #CO = ascii.read('C:/Users/seven/OneDrive/Documents/DustSurvey/CO' + str(l) + 'p' + str(b) + '.txt')
    
    #CODarkDustmaps/OH67basesubbed/OH119070p2800_67basesub.txt
    #CODarkDustmaps/HI/HI119380p3967.txt
    #CODarkDustmaps/CO/CO119070p2800.txt

    OH67 = ascii.read('OH67basesubbed/OH' + str(l) + 'p' + str(b) + '_67basesub.txt')
    OH65 = ascii.read('OH65basesubbed/OH' + str(l) + 'p' + str(b) + '_65basesub.txt')
    HI = ascii.read('HI/HI' + str(l) + 'p' + str(b) + '.txt', data_start = 13, data_end = 167)
    CO = ascii.read('CO/CO' + str(l) + 'p' + str(b) + '.txt')
    
    resultsTable = pd.DataFrame(np.array([[0, 
                                         0, 
                                         0,
                                         0,
                                         0,
                                         0,
                                         0,
                                         0, 
                                         0,
                                         0,
                                         0,
                                         0,
                                         0,
                                         0]]),
                               columns=['l (deg)', 
                                        'b (deg)', 
                                        'line',
                                        'comp',
                                        'T_peak (K)',
                                        'T_error (K)', 
                                        'CenterVel (km/s)',
                                        'CenterVel_error (km/s)',
                                        'FWHM (km/s)',
                                        'FWHM error (km/s)',
                                        'integral',
                                        'integral error',
                                        'Cloud',
                                        'Upper Limit'
                                       ])
    velHI = HI['col1']
    TaHI = HI['col2']
    vel67 = OH67['col1']
    Ta67 = OH67['col2']
    vel65 = OH65['col1']
    Ta65 = OH65['col2']
    velCO = CO['col1']
    TaCO = CO['col2']
    sigT67 = np.std(Ta67[np.where((-100 <= vel67) & (-25>vel67))])
    sigT65 = np.std(Ta65[np.where((-100 <= vel65) & (-25>vel65))])
    sigTCO = np.std(TaCO[np.where(velCO< -15)])
    sigTHI = np.std(TaHI[np.where(velHI< -50)])
    dVHI = velHI[1]-velHI[0]
    dV67 = vel67[1]-vel67[0]
    dV65 = vel65[1]-vel65[0]
    dVCO = velCO[1]-velCO[0]    
    
    size67 = len(vel67)
    size65 = len(vel65)
    sizeHI = len(velHI)
    sizeCO = len(velCO)
    
    deltaV67 = vel67[1]-vel67[0]
    deltaV65 = vel65[1]-vel65[0]
    deltaVHI = velHI[1]-velHI[0]
    deltaVCO = velCO[1]-velCO[0]
    
    sig = 0.003 #conventional 3mK sigma

    #fit HI data
    if n_HI == 1:

        parametersHI, covarianceHI = curve_fit(Gauss, velHI, TaHI, HI_p0, bounds = HI_bounds)

        fitHIh = [parametersHI[0]]
        fitHIc = [parametersHI[1]]
        fitHIw = [np.abs(parametersHI[2])]
        perrHI = np.sqrt(np.diag(covarianceHI))
        errorHIh = [perrHI[0]]
        errorHIc = [perrHI[1]]
        errorHIw = [perrHI[2]]
                          
        v1HI = parametersHI[1] - parametersHI[2]*1.5
        v2HI = parametersHI[1] + parametersHI[2]*1.5 
                          
        intrang1 = np.where((velHI<v2HI) & (velHI>v1HI))
                          
        chan1HI = len(velHI[intrang1])
        
        #intHI_1 =  intfac * parametersHI[0] * parametersHI[2]
        intHI_1 = np.sum(TaHI[intrang1])*dVHI
        intHI = [intHI_1]
                          
        #interrHI_1 = np.sqrt(perrHI[0]**2 + perrHI[2]**2)
        interrHI_1 = np.sqrt(chan1HI)*sigTHI*dVHI
        interrHI = [interrHI_1]
        
        UpperLimHI = ['F'] #HI has no upper limits
        
        for i in range(n_HI):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        'HI',
                                                        i+1,
                                                        fitHIh[i],
                                                        errorHIh[i],
                                                        fitHIc[i],
                                                        errorHIc[i],
                                                        fitHIw[i],
                                                        errorHIw[i],
                                                        intHI[i],
                                                        interrHI[i],
                                                        Cloud,
                                                        UpperLimHI[i]
                                                        ]
    if n_HI == 2:
        
        HI_p0 = np.tile(HI_p0, n_HI)
        HI_bounds = np.tile(HI_bounds, n_HI)
        parametersHI, covarianceHI = curve_fit(doubleGauss, velHI, TaHI, HI_p0, bounds = HI_bounds)

        fitHIh = [parametersHI[0], parametersHI[3]]
        fitHIc = [parametersHI[1], parametersHI[4]]
        fitHIw = [np.abs(parametersHI[2]), np.abs(parametersHI[5])]
        perrHI = np.sqrt(np.diag(covarianceHI))
        errorHIh = [perrHI[0], perrHI[3]]
        errorHIc = [perrHI[1], perrHI[4]]
        errorHIw = [perrHI[2], perrHI[5]]
        
        #first component integral range
        v1HI = parametersHI[1] - parametersHI[2]*1.5
        v2HI = parametersHI[1] + parametersHI[2]*1.5 
        #second component integral range
        v3HI = parametersHI[4] - parametersHI[5]*1.5
        v4HI = parametersHI[4] + parametersHI[5]*1.5
                          
        intrang1 = np.where((velHI<v2HI) & (velHI>v1HI))
        intrang2 = np.where((velHI<v4HI) & (velHI>v3HI))
        
        #channel number bounded
        chan1HI = len(velHI[intrang1])
        chan2HI = len(velHI[intrang2])
                          
        intHI_1 = np.sum(TaHI[intrang1])*dVHI
        intHI_2 = np.sum(TaHI[intrang2])*dVHI
        intHI = [intHI_1, intHI_2]
                          
        interrHI_1 = np.sqrt(chan1HI) * sigTHI * dVHI
        interrHI_2 = np.sqrt(chan2HI) * sigTHI * dVHI
        interrHI = [interrHI_1, interrHI_2]
        
        UpperLimHI = ['F', 'F'] #HI has no upper limits
        
        for i in range(n_HI):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        'HI',
                                                        i+1,
                                                        fitHIh[i],
                                                        errorHIh[i],
                                                        fitHIc[i],
                                                        errorHIc[i],
                                                        fitHIw[i],
                                                        errorHIw[i],
                                                        intHI[i],
                                                        interrHI[i],
                                                        Cloud,
                                                        UpperLimHI[i]
                                                        ]
            
    if n_HI == 3:
        
        HI_p0 = np.tile(HI_p0, n_HI)
        HI_bounds = np.tile(HI_bounds, n_HI)
        parametersHI, covarianceHI = curve_fit(tripleGauss, velHI, TaHI, HI_p0, bounds = HI_bounds)
        
        fitHIh = [parametersHI[0], parametersHI[3], parametersHI[6]]
        fitHIc = [parametersHI[1], parametersHI[4], parametersHI[7]]
        fitHIw = [np.abs(parametersHI[2]), np.abs(parametersHI[5]), np.abs(parametersHI[8])]
        perrHI = np.sqrt(np.diag(covarianceHI))
        errorHIh = [perrHI[0], perrHI[3], perrHI[6]]
        errorHIc = [perrHI[1], perrHI[4], perrHI[7]]
        errorHIw = [perrHI[2], perrHI[5], perrHI[8]]
        
        #first component integral range
        v1HI = parametersHI[1] - parametersHI[2]*1.5
        v2HI = parametersHI[1] + parametersHI[2]*1.5 
        #second component integral range
        v3HI = parametersHI[4] - parametersHI[5]*1.5
        v4HI = parametersHI[4] + parametersHI[5]*1.5
        #third component integral range\
        v5HI = parametersHI[7]-parametersHI[8]*1.5
        v6HI = parametersHI[7]+parametersHI[8]*1.5
        
        #integral ranges
        intrang1 = np.where((velHI<v2HI) & (velHI>v1HI))
        intrang2 = np.where((velHI<v4HI) & (velHI>v3HI))
        intrang3 = np.where((velHI<v6HI) & (velHI>v5HI))
        
        #channel number bounded
        chan1HI = len(velHI[intrang1])
        chan2HI = len(velHI[intrang2])
        chan3HI = len(velHI[intrang3])
        
        #component integrals
        intHI_1 = np.sum(TaHI[intrang1])*dVHI
        intHI_2 = np.sum(TaHI[intrang2])*dVHI
        intHI_3 = np.sum(TaHI[intrang3])*dVHI
        intHI = [intHI_1, intHI_2, intHI_3]
        
        #component integral errors
        interrHI_1 = np.sqrt(chan1HI) * sigTHI * dVHI
        interrHI_2 = np.sqrt(chan2HI) * sigTHI * dVHI
        interrHI_3 = np.sqrt(chan3HI) * sigTHI * dVHI
        interrHI = [interrHI_1, interrHI_2, interrHI_3]
        UpperLimHI = ['F', 'F', 'F'] #HI has no upper limits

        for i in range(n_HI):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        'HI',
                                                        i+1,
                                                        fitHIh[i],
                                                        errorHIh[i],
                                                        fitHIc[i],
                                                        errorHIc[i],
                                                        fitHIw[i],
                                                        errorHIw[i],
                                                        intHI[i],
                                                        interrHI[i],
                                                        Cloud,
                                                        UpperLimHI[i]
                                                        ]
    #fit 67 data
    
    if n_67 == 1:
        parameters67, covariance67 = curve_fit(Gauss, vel67, Ta67, OH67_p0, bounds = OH67_bounds, sigma = [sig] * size67, absolute_sigma = True)
        
        fit67h = [parameters67[0]]
        fit67c = [parameters67[1]]
        fit67w = [np.abs(parameters67[2])]
        perr67 = np.sqrt(np.diag(covariance67))
        error67h = [perr67[0]]
        error67c = [perr67[1]]
        error67w = [perr67[2]]
        
        #1st comp int rang
        v167 = parameters67[1] - parameters67[2]*1.5
        v267 = parameters67[1] + parameters67[2]*1.5    
        
        #integral range
        intrang167 = np.where((vel67<v267) & (vel67>v167))
        #channels in 1st comp
        chan167 = len(vel67[intrang167])

        #integral
        int67_1 = np.sum(Ta67[intrang167])*dV67
        int67 = [int67_1]
        intuplim67 = sigT67*parameters67[2]                

        #integral error
        interr67_1 = np.sqrt(chan167)*sigT67*dV67
        interr67 = [interr67_1]
        interruplim67 = sigT67*perr67[2]
        

        
        r = n_HI - n_67
        
        
        if r == 0:
            UpperLim67 = ['F'] #1 gauss
            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            '67',
                                                            i+1,
                                                            fit67h[i],
                                                            error67h[i],
                                                            fit67c[i],
                                                            error67c[i],
                                                            fit67w[i],
                                                            error67w[i],
                                                            int67[i],
                                                            interr67[i],
                                                            Cloud,
                                                            UpperLim67[i]
                                                            ]
        elif r == 1:
            fit67h = [parameters67[0], sigT67]
            fit67c = [parameters67[1], deltaV67]
            fit67w = [np.abs(parameters67[2]), deltaV67]
            error67h = [perr67[0], perrHI[3]]
            error67c = [perr67[1], perrHI[4]]
            error67w = [perr67[2], perrHI[5]]
            int67 = [int67_1, intuplim67] #What do I put for upper limits for integrals?
            interr67 = [interr67_1, interruplim67]
            UpperLim67 = ['F', 'T'] #1 gauss 1 upperlim
            
            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            '67',
                                                            i+1,
                                                            fit67h[i],
                                                            error67h[i],
                                                            fit67c[i],
                                                            error67c[i],
                                                            fit67w[i],
                                                            error67w[i],
                                                            int67[i],
                                                            interr67[i],
                                                            Cloud,
                                                            UpperLim67[i]
                                                            ]       
        elif r == 2:
            fit67h = [parameters67[0], sigT67, sigT67]
            fit67c = [parameters67[1], deltaV67, deltaV67]
            fit67w = [np.abs(parameters67[2]), deltaV67, deltaV67]
            error67h = [perr67[0], perrHI[3], perrHI[6]]
            error67c = [perr67[1], perrHI[4], perrHI[7]]
            error67w = [perr67[2], perrHI[5], perrHI[8]]
            int67 = [int67_1, intuplim67, intuplim67]
            interr67 = [interr67_1, interruplim67, interruplim67]
            UpperLim67 = ['F', 'T', 'T'] #1 gauss 2 uplim
            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            '67',
                                                            i+1,
                                                            fit67h[i],
                                                            error67h[i],
                                                            fit67c[i],
                                                            error67c[i],
                                                            fit67w[i],
                                                            error67w[i],
                                                            int67[i],
                                                            interr67[i],
                                                            Cloud,
                                                            UpperLim67[i]
                                                            ]     
        
    if n_67 == 2:
        
        OH67_p0 = np.tile(OH67_p0, n_67)
        OH67_bounds = np.tile(OH67_bounds, n_67)
        parameters67, covariance67 = curve_fit(doubleGauss, vel67, Ta67, OH67_p0, bounds = OH67_bounds, sigma = [sig] * size67, absolute_sigma = True)

        fit67h = [parameters67[0], parameters67[3]]
        fit67c = [parameters67[1], parameters67[4]]
        fit67w = [np.abs(parameters67[2]), np.abs(parameters67[5])]
        perr67 = np.sqrt(np.diag(covariance67))
        error67h = [perr67[0], perr67[3]]
        error67c = [perr67[1], perr67[4]]
        error67w = [perr67[2], perr67[5]]
        
        #int range by component
        v167 = parameters67[1] - parameters67[2]*1.5
        v267 = parameters67[1] + parameters67[2]*1.5    

        v367 = parameters67[4]-parameters67[5]*1.5
        v467 = parameters67[4]+parameters67[5]*1.5

        
        #integral range
        intrang167 = np.where((vel67<v267) & (vel67>v167))
        intrang267 = np.where((vel67<v467) & (vel67>v367))
        #channels per comp
        chan167 = len(vel67[intrang167])
        chan267 = len(vel67[intrang267])
    

        int67_1 = np.sum(Ta67[intrang167])*dV67
        int67_2 = np.sum(Ta67[intrang267])*dV67
        int67 = [int67_1, int67_2]
        intuplim67 = sigT67*parameters67[2]                     

        interr67_1 = np.sqrt(chan167)*sigT67*dV67
        interr67_2 = np.sqrt(chan267)*sigT67*dV67
        interr67 = [interr67_1, interr67_2]
        interruplim67 = sigT67*perr67[2]
        
        r = n_HI - n_67

        if r == 0:
            UpperLim67 = ['F', 'F'] #upper limits false here because 2 gaussian components
            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            '67',
                                                            i+1,
                                                            fit67h[i],
                                                            error67h[i],
                                                            fit67c[i],
                                                            error67c[i],
                                                            fit67w[i],
                                                            error67w[i],
                                                            int67[i],
                                                            interr67[i],
                                                            Cloud,
                                                            UpperLim67[i]
                                                            ]
                
        elif r==1:
            fit67h = [parameters67[0], parameters67[3], sigT67]
            fit67c = [parameters67[1], parameters67[4], deltaV67]
            fit67w = [np.abs(parameters67[2]), np.abs(parameters67[5]), deltaV67]
            error67h = [perr67[0],  perr67[3], perrHI[6]]
            error67c = [perr67[1],  perr67[4], perrHI[7]]
            error67w = [perr67[2],  perr67[5], perrHI[8]]
            UpperLim67 = ['F', 'F', 'T'] #2 gauss 1 uplim
            int67 = [int67_1, int67_2, intuplim67]
            interr67 = [interr67_1, interr67_2, interruplim67]
            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            '67',
                                                            i+1,
                                                            fit67h[i],
                                                            error67h[i],
                                                            fit67c[i],
                                                            error67c[i],
                                                            fit67w[i],
                                                            error67w[i],
                                                            int67[i],
                                                            interr67[i],
                                                            Cloud,
                                                            UpperLim67[i]
                                                            ]
    if n_67 == 3:
        
        OH67_p0 = np.tile(OH67_p0, n_67)
        OH67_bounds = np.tile(OH67_bounds, n_67)
        parameters67, covariance67 = curve_fit(tripleGauss, vel67, Ta67, OH67_p0, bounds = OH67_bounds, sigma = [sig] * size65, absolute_sigma = True) 
    
        fit67h = [parameters67[0], parameters67[3], parameters67[6]]
        fit67c = [parameters67[1], parameters67[4], parameters67[7]]
        fit67w = [np.abs(parameters67[2]), np.abs(parameters67[5]), np.abs(parameters67[8])]
        perr67 = np.sqrt(np.diag(covariance67))
        error67h = [perr67[0], perr67[3], perr67[6]]
        error67c = [perr67[1], perr67[4], perr67[7]]
        error67w = [perr67[2], perr67[5], perr67[8]]
                         
        #int range by component, 1st number dictates component
        v167 = parameters67[1] - parameters67[2]*1.5
        v267 = parameters67[1] + parameters67[2]*1.5    

        v367 = parameters67[4]-parameters67[5]*1.5
        v467 = parameters67[4]+parameters67[5]*1.5
                         
        v567 = parameters67[7]-parameters67[8]*1.5
        v667 = parameters67[7]+parameters67[8]*1.5

        #integral range, 1st number dictates component
        intrang167 = np.where((vel67<v267) & (vel67>v167))
        intrang267 = np.where((vel67<v467) & (vel67>v367))
        intrang367 = np.where((vel67<v667) & (vel67>v567))
        #channels per comp, 1st number dictates component
        chan167 = len(vel67[intrang167])
        chan267 = len(vel67[intrang267])
        chan367 = len(vel67[intrang367])
        
        #integrals, number after underscore dictates component
        int67_1 = np.sum(Ta67[intrang167])*dV67
        int67_2 = np.sum(Ta67[intrang267])*dV67
        int67_3 = np.sum(Ta67[intrang267])*dV67
        int67 = [int67_1, int67_2, int67_3]
                          
        interr67_1 = np.sqrt(chan167)*sigT67*dV67
        interr67_2 = np.sqrt(chan267)*sigT67*dV67
        interr67_3 = np.sqrt(chan367)*sigT67*dV67
        interr67 = [interr67_1, interr67_2, interr67_3]

        for i in range(n_67):
            UpperLim67 = ['F','F','F'] #upper lim false because 3 gauss components
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        '67',
                                                        i+1,
                                                        fit67h[i],
                                                        error67h[i],
                                                        fit67c[i],
                                                        error67c[i],
                                                        fit67w[i],
                                                        error67w[i],
                                                        int67[i],
                                                        interr67[i],
                                                        Cloud,
                                                        UpperLim67[i]
                                                        ]        
    #fit 65 data
    if n_65 == 1:
        parameters65, covariance65 = curve_fit(Gauss, vel65, Ta65, OH65_p0, bounds = OH65_bounds, sigma = [sig] * size65, absolute_sigma = True)
        
        fit65h = [parameters65[0]]
        fit65c = [parameters65[1]]
        fit65w = [np.abs(parameters65[2])]
        perr65 = np.sqrt(np.diag(covariance65))
        error65h = [perr65[0]]
        error65c = [perr65[1]]
        error65w = [perr65[2]]
                          
        #1st comp int rang
        v165 = parameters65[1] - parameters65[2]*1.5
        v265 = parameters65[1] + parameters65[2]*1.5    
        
        #integral range
        intrang165 = np.where((vel65<v265) & (vel65>v165))
        #channels in 1st comp
        chan165 = len(vel65[intrang165])

        #integral
        int65_1 = np.sum(Ta65[intrang165])*dV65
        int65 = [int65_1]
        intuplim65 = sigT65*parameters65[2]   
                          
        #integral error
        interr65_1 = np.sqrt(chan165)*sigT65*dV65
        interr65 = [interr65_1]
        interruplim65 = sigT65*perr65[2]
        
        s = n_HI - n_65
        
        if s == 0:
            UpperLim65 = ['F'] #1 gaussian comp

            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            '65',
                                                            i+1,
                                                            fit65h[i],
                                                            error65h[i],
                                                            fit65c[i],
                                                            error65c[i],
                                                            fit65w[i],
                                                            error65w[i],
                                                            int65[i],
                                                            interr65[i],
                                                            Cloud,
                                                            UpperLim65[i]
                                                            ]
        elif s == 1:
            fit65h = [parameters65[0], sigT65]
            fit65c = [parameters65[1], parameters67[1]]
            fit65w = [np.abs(parameters65[2]), np.abs(parameters67[2])]
            error65h = [perr65[0], perrHI[3]]
            error65c = [perr65[1], perrHI[4]]
            error65w = [perr65[2], perrHI[5]]
            UpperLim65 = ['F', 'T'] #1 gauss 1 UpLim
            int65 = [int65_1, intuplim65]
            interr65 = [interr65_1, interruplim65]
            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            '65',
                                                            i+1,
                                                            fit65h[i],
                                                            error65h[i],
                                                            fit65c[i],
                                                            error65c[i],
                                                            fit65w[i],
                                                            error65w[i],
                                                            int65[i],
                                                            interr65[i],
                                                            Cloud,
                                                            UpperLim65[i]
                                                            ]       
        elif s == 2:
            fit65h = [parameters65[0], sigT65, sigT65]
            fit65c = [parameters65[1], parameters67[1], parameters67[1]]
            fit65w = [np.abs(parameters65[2]), np.abs(parameters67[2]), np.abs(parameters67[2])]
            error65h = [perr65[0], perrHI[3], perrHI[6]]
            error65c = [perr65[1], perrHI[4], perrHI[7]]
            error65w = [perr65[2], perrHI[5], perrHI[8]]
            UpperLim65 = ['F', 'T', 'T'] #1 gauss 2 uplim
            int65 = [int65_1, intuplim65, intuplim65]
            interr65 = [interr65_1, interruplim65, interruplim65]
            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            '65',
                                                            i+1,
                                                            fit65h[i],
                                                            error65h[i],
                                                            fit65c[i],
                                                            error65c[i],
                                                            fit65w[i],
                                                            error65w[i],
                                                            int65[i],
                                                            interr65[i],
                                                            Cloud,
                                                            UpperLim65[i]
                                                            ]     
    if n_65 == 2:
        
        OH65_p0 = np.tile(OH65_p0, n_65)
        OH65_bounds = np.tile(OH65_bounds, n_65)
        parameters65, covariance65 = curve_fit(doubleGauss, vel65, Ta65, OH65_p0, bounds = OH65_bounds, sigma = [sig] * size65, absolute_sigma = True)
    
        fit65h = [parameters65[0], parameters65[3]]
        fit65c = [parameters65[1], parameters65[4]]
        fit65w = [np.abs(parameters65[2]), np.abs(parameters65[4])]
        perr65 = np.sqrt(np.diag(covariance65))
        print(perr65)
        error65h = [perr65[0], perr65[3]]
        error65c = [perr65[1], perr65[4]]
        error65w = [perr65[2], perr65[5]]
        
        #int range by component
        v165 = parameters65[1] - parameters65[2]*1.5
        v265 = parameters65[1] + parameters65[2]*1.5    

        v365 = parameters65[4]-parameters65[5]*1.5
        v465 = parameters65[4]+parameters65[5]*1.5

        #integral range
        intrang165 = np.where((vel65<v265) & (vel65>v165))
        intrang265 = np.where((vel65<v465) & (vel65>v365))
        #channels per comp
        chan165 = len(vel65[intrang165])
        chan265 = len(vel65[intrang265])
    

        int65_1 = np.sum(Ta65[intrang165])*dV65
        int65_2 = np.sum(Ta65[intrang265])*dV65
        int65 = [int65_1, int65_2]
        intuplim65 = sigT65*parameters65[2]
                          
        #interrHI_1 = np.sqrt(perrHI[0]**2 + perrHI[2]**2)
        interr65_1 = np.sqrt(chan165)*sigT65*dV65
        interr65_2 = np.sqrt(chan265)*sigT65*dV65
        interr65 = [interr65_1, interr65_2]
        interruplim65 = sigT65*perr65[2]
        
        s = n_HI - n_65
        
        if s == 0:
            UpperLim65 = ['F', 'F'] #1 gauss comp

            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            '65',
                                                            i+1,
                                                            fit65h[i],
                                                            error65h[i],
                                                            fit65c[i],
                                                            error65c[i],
                                                            fit65w[i],
                                                            error65w[i],
                                                            int65[i],
                                                            interr65[i],
                                                            Cloud,
                                                            UpperLim65[i]
                                                            ]
        elif s == 1:
            fit65h = [parameters65[0], parameters65[3], sigT65]
            fit65c = [parameters65[1], parameters65[4], parameters67[1]]
            fit65w = [np.abs(parameters65[2]), np.abs(parameters65[4]), parameters67[2]]
            error65h = [perr65[0], perr65[3], perrHI[6]]
            error65c = [perr65[1], perr65[4], perrHI[7]]
            error65w = [perr65[2], perr65[5], perrHI[8]]
            UpperLim65 = ['F', 'F', 'T'] #2 gauss 1 uplim
            int65 = [int65_1, int65_2, intuplim65]
            interr65 = [interr65_1, interr65_2, intuplim65]
            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            '65',
                                                            i+1,
                                                            fit65h[i],
                                                            error65h[i],
                                                            fit65c[i],
                                                            error65c[i],
                                                            fit65w[i],
                                                            error65w[i],
                                                            int65[i],
                                                            interr65[i],
                                                            Cloud,
                                                            UpperLim65[i]
                                                            ]       
    if n_65 == 3:
        
        OH65_p0 = np.tile(OH65_p0, n_65)
        OH65_bounds = np.tile(OH65_bounds, n_65)
        parameters65, covariance65 = curve_fit(tripleGauss, vel65, Ta65, OH65_p0, bounds = OH65_bounds, sigma = [sig] * size65, absolute_sigma = True)    
        
        fit65h = [parameters65[0], parameters65[3], parameters65[6]]
        fit65c = [parameters65[1], parameters65[4], parameters65[7]]
        fit65w = [np.abs(parameters65[2]), np.abs(parameters65[4]), np.abs(parameters65[8])]
        perr65 = np.sqrt(np.diag(covariance65))
        error65h = [perr65[0], perr65[3], perr65[6]]
        error65c = [perr65[1], perr65[4], perr65[7]]
        error65w = [perr65[2], perr65[5], perr65[8]]

        #int range by component, 1st number dictates component
        v165 = parameters65[1] - parameters65[2]*1.5
        v265 = parameters65[1] + parameters65[2]*1.5    

        v365 = parameters65[4]-parameters65[5]*1.5
        v465 = parameters65[4]+parameters65[5]*1.5
                         
        v565 = parameters65[7]-parameters65[8]*1.5
        v665 = parameters65[7]+parameters65[8]*1.5
                         
        #integral range, 1st number dictates component
        intrang165 = np.where((vel65<v265) & (vel65>v165))
        intrang265 = np.where((vel65<v465) & (vel65>v365))
        intrang365 = np.where((vel65<v665) & (vel65>v565))
        #channels per comp, 1st number dictates component
        chan165 = len(vel65[intrang165])
        chan265 = len(vel65[intrang265])
        chan365 = len(vel65[intrang365])
        
        #integrals, number after underscore dictates component
        int65_1 = np.sum(Ta65[intrang165])*dV65
        int65_2 = np.sum(Ta65[intrang265])*dV65
        int65_3 = np.sum(Ta65[intrang265])*dV65
        int65 = [int65_1, int65_2, int65_3]
                          
        #interrHI_1 = np.sqrt(perrHI[0]**2 + perrHI[2]**2)
        interr65_1 = np.sqrt(chan165)*sigT65*dV65
        interr65_2 = np.sqrt(chan265)*sigT65*dV65
        interr65_3 = np.sqrt(chan365)*sigT65*dV65
        interr65 = [interr65_1, interr65_2, interr65_3]

        
        for i in range(n_65):
            UpperLim65 = ['F', 'F', 'F'] #3 gauss comps
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        '65',
                                                        i+1,
                                                        fit65h[i],
                                                        error65h[i],
                                                        fit65c[i],
                                                        error65c[i],
                                                        fit65w[i],
                                                        error65w[i],
                                                        int65[i],
                                                        interr65[i],
                                                        Cloud,
                                                        UpperLim65[i]
                                                        ]
    #fit CO data
    if n_CO == 1:
        parametersCO, covarianceCO = curve_fit(Gauss, velCO, TaCO, CO_p0,bounds =  CO_bounds)
            
        fitCOh = [parametersCO[0]]
        fitCOc = [parametersCO[1]]
        fitCOw = [np.abs(parametersCO[2])]
        perrCO = np.sqrt(np.diag(covarianceCO))
        errorCOh = [perrCO[0]]
        errorCOc = [perrCO[1]]
        errorCOw = [perrCO[2]]
                         
        #1st comp int rang
        v1CO = parametersCO[1] - parametersCO[2]*1.5
        v2CO = parametersCO[1] + parametersCO[2]*1.5    
        
        #integral range
        intrang1CO = np.where((velCO<v2CO) & (velCO>v1CO))
        #channels in 1st comp
        chan1CO = len(velCO[intrang1CO])

        #integral
        intCO_1 = np.sum(TaCO[intrang1CO])*dVCO
        intCO = [intCO_1]
        intuplimCO = sigTCO*parametersCO[2]
                          
        #integral error
        interrCO_1 = np.sqrt(chan1CO)*sigTCO*dVCO
        interrCO = [interrCO_1]
        interruplimCO = sigTCO*perrCO[2]
   
        q = n_HI - n_CO
        
        if q == 0:
            UpperLimCO = ['F'] #1gauss comp

            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            'CO',
                                                            i+1,
                                                            fitCOh[i],
                                                            errorCOh[i],
                                                            fitCOc[i],
                                                            errorCOc[i],
                                                            fitCOw[i],
                                                            errorCOw[i],
                                                            intCO[i],
                                                            interrCO[i],
                                                            Cloud,
                                                            UpperLimCO[i]
                                                            ]
        elif q == 1:
            fitCOh = [parametersCO[0], sigTCO]
            fitCOc = [parametersCO[1], parameters65[1]]
            fitCOw = [np.abs(parametersCO[2]), parameters65[2]]
            errorCOh = [perrCO[0], perrHI[3]]
            errorCOc = [perrCO[1], perrHI[4]]
            errorCOw = [perrCO[2], perrHI[5]]
            UpperLimCO = ['F', 'T'] #1 gauss, 1 uplim
            intCO = [intCO_1, intuplimCO]
            interrCO = [interrCO_1, interruplimCO]
            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            'CO',
                                                            i+1,
                                                            fitCOh[i],
                                                            errorCOh[i],
                                                            fitCOc[i],
                                                            errorCOc[i],
                                                            fitCOw[i],
                                                            errorCOw[i],
                                                            intCO[i],
                                                            interrCO[i],
                                                            Cloud,
                                                            UpperLimCO[i]
                                                            ]       
        elif q == 2:
            fitCOh = [parametersCO[0], sigTCO, sigTCO]
            fitCOc = [parametersCO[1], parameters65[1], parameters65[1]]
            fitCOw = [np.abs(parametersCO[2]), parameters65[2], parameters65[2]]
            errorCOh = [perrCO[0], perrHI[3], perrHI[6]]
            errorCOc = [perrCO[1], perrHI[4], perrHI[7]]
            errorCOw = [perrCO[2], perrHI[5], perrHI[8]]
            UpperLimCO = ['F', 'T', 'T'] #1 gauss 2 uplim
            intCO = [intCO_1, intuplimCO, intuplimCO]
            interrCO = [interrCO_1, interruplimCO, interruplimCO]
            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            'CO',
                                                            i+1,
                                                            fitCOh[i],
                                                            errorCOh[i],
                                                            fitCOc[i],
                                                            errorCOc[i],
                                                            fitCOw[i],
                                                            errorCOw[i],
                                                            intCO[i],
                                                            interrCO[i],
                                                            Cloud,
                                                            UpperLimCO[i]
                                                            ]    
    if n_CO == 2:
        
        CO_p0 = np.tile(CO_p0, n_CO)
        CO_bounds = np.tile(CO_bounds, n_CO)
        parametersCO, covarianceCO = curve_fit(doubleGauss, velCO, TaCO, CO_p0, bounds = CO_bounds)
    
        fitCOh = [parametersCO[0], parametersCO[3]]
        fitCOc = [parametersCO[1], parametersCO[4]]
        fitCOw = [np.abs(parametersCO[2]), np.abs(parametersCO[5])] 
        perrCO = np.sqrt(np.diag(covarianceCO))
        errorCOh = [perrCO[0], perrCO[3]]
        errorCOc = [perrCO[1], perrCO[4]]
        errorCOw = [perrCO[2], perrCO[5]]
        
        #int range by component
        v1CO = parametersCO[1] - parametersCO[2]*1.5
        v2CO = parametersCO[1] + parametersCO[2]*1.5    

        v3CO = parametersCO[4]-parametersCO[5]*1.5
        v4CO = parametersCO[4]+parametersCO[5]*1.5

        #integral range
        intrang1CO = np.where((velCO<v2CO) & (velCO>v1CO))
        intrang2CO = np.where((velCO<v4CO) & (velCO>v3CO))
        #channels per comp
        chan1CO = len(velCO[intrang1CO])
        chan2CO = len(velCO[intrang2CO])
    

        intCO_1 = np.sum(TaCO[intrang1CO])*dVCO
        intCO_2 = np.sum(TaCO[intrang2CO])*dVCO
        intCO = [intCO_1, intCO_2]
        intuplimCO = sigTCO * parametersCO[2]
                          
        #interrHI_1 = np.sqrt(perrHI[0]**2 + perrHI[2]**2)
        interrCO_1 = np.sqrt(chan1CO)*sigTCO*dVCO
        interrCO_2 = np.sqrt(chan2CO)*sigTCO*dVCO
        interrCO = [interrCO_1, interrCO_2]
        interruplimCO = sigTCO*perrCO[2]
        
        q = n_HI - n_CO
        
        if q == 0:
            UpperLimCO = ['F', 'F'] #2 gauss

            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            'CO',
                                                            i+1,
                                                            fitCOh[i],
                                                            errorCOh[i],
                                                            fitCOc[i],
                                                            errorCOc[i],
                                                            fitCOw[i],
                                                            errorCOw[i],
                                                            intCO[i],
                                                            interrCO[i],
                                                            Cloud,
                                                            UpperLimCO[i]
                                                            ]
        elif q == 1:
            fitCOh = [parametersCO[0], parametersCO[3], sigTCO]
            fitCOc = [parametersCO[1], parametersCO[4], parameters65[1]]
            fitCOw = [np.abs(parametersCO[2]), np.abs(parametersCO[5]), parameters65[2]]
            errorCOh = [perrCO[0], perrCO[3], perrHI[6]]
            errorCOc = [perrCO[1], perrCO[4], perrHI[7]]
            errorCOw = [perrCO[2], perrCO[5], perrHI[8]]
            UpperLimCO = ['F', 'F', 'T'] # 2 gauss 1 uplim
            intCO = [intCO_1, intCO_2, intuplimCO]
            interrCO = [interrCO_1, interrCO_1, interruplimCO]
            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            'CO',
                                                            i+1,
                                                            fitCOh[i],
                                                            errorCOh[i],
                                                            fitCOc[i],
                                                            errorCOc[i],
                                                            fitCOw[i],
                                                            errorCOw[i],
                                                            intCO[i],
                                                            interrCO[i],
                                                            Cloud,
                                                            UpperLimCO[i]
                                                            ]     
    if n_CO == 3:
        
        CO_p0 = np.tile(CO_p0, n_CO)
        CO_bounds = np.tile(CO_bounds, n_CO)
        parametersCO, covarianceCO = curve_fit(tripleGauss, velCO, TaCO, CO_p0, bounds = CO_bounds) 
            
        fitCOh = [parametersCO[0], parametersCO[3], parametersCO[6]]
        fitCOc = [parametersCO[1], parametersCO[4], parametersCO[7]]
        fitCOw = [np.abs(parametersCO[2]), np.abs(parametersCO[5]), np.abs(parametersCO[8])] 
        perrCO = np.sqrt(np.diag(covarianceCO))
        errorCOh = [perrCO[0], perrCO[3], perrCO[6]]
        errorCOc = [perrCO[1], perrCO[4], perrCO[7]]
        errorCOw = [perrCO[2], perrCO[5], perrCO[8]]
                         
        #int range by component, 1st number dictates component
        v1CO = parametersCO[1] - parametersCO[2]*1.5
        v2CO = parametersCO[1] + parametersCO[2]*1.5    

        v3CO = parametersCO[4]-parametersCO[5]*1.5
        v4CO = parametersCO[4]+parametersCO[5]*1.5
                         
        v5CO = parametersCO[7]-parametersCO[8]*1.5
        v6CO = parametersCO[7]+parametersCO[8]*1.5
                         
        #integral range, 1st number dictates component
        intrang1CO = np.where((velCO<v2CO) & (velCO>v1CO))
        intrang2CO = np.where((velCO<v4CO) & (velCO>v3CO))
        intrang3CO = np.where((velCO<v6CO) & (velCO>v5CO))
        #channels per comp, 1st number dictates component
        chan1CO = len(velCO[intrang1CO])
        chan2CO = len(velCO[intrang2CO])
        chan3CO = len(velCO[intrang3CO])
        
        #integrals, number after underscore dictates component
        intCO_1 = np.sum(TaCO[intrang1CO])*dVCO
        intCO_2 = np.sum(TaCO[intrang2CO])*dVCO
        intCO_3 = np.sum(TaCO[intrang2CO])*dVCO
        intCO = [intCO_1, intCO_2, intCO_3]
                          
        #interrHI_1 = np.sqrt(perrHI[0]**2 + perrHI[2]**2)
        interrCO_1 = np.sqrt(chan1CO)*sigTCO*dVCO
        interrCO_2 = np.sqrt(chan2CO)*sigTCO*dVCO
        interrCO_3 = np.sqrt(chan3CO)*sigTCO*dVCO
        interrCO = [interrCO_1, interrCO_2, interrCO_3]
        
        for i in range(n_CO):
            UpperLimCO = ['F', 'F', 'F'] #3 gauss
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        'CO',
                                                        i+1,
                                                        fitCOh[i],
                                                        errorCOh[i],
                                                        fitCOc[i],
                                                        errorCOc[i],
                                                        fitCOw[i],
                                                        errorCOw[i],
                                                        intCO[i],
                                                        interrCO[i],
                                                        Cloud,
                                                        UpperLimCO[i]
                                                        ] 
    resultsTable = resultsTable.drop(0)
    return resultsTable'''

SyntaxError: invalid syntax (3568083628.py, line 5)

In [ ]:
def fit_coord(l, b, n_HI = 1, n_65 = 1, n_67 = 1, n_CO = 1, HI_p0 = [1, 0, 1], OH65_p0 = [0.03, 0, 4], OH67_p0 = [0.03, 0, 4], CO_p0 = [1, 0, 1], HI_bounds = (-np.inf, np.inf), OH65_bounds = (-np.inf, np.inf), OH67_bounds = (-np.inf, np.inf), CO_bounds = (-np.inf, np.inf), Cloud = []):
    
    # Setup a pandas dataframe to save all the variables we'd like
    
    '''Purpose:
    This function fits multiple gaussians to HI, CO and HI data and returns the parameters...
    
    Parameters:
    l: galactic latitude, int
    b: galactic lontitude, int
    HIdata: np.array of HI data in Tb, vel
    OHdata: np.array of OH data in Tb, vel
    ...

    Returns a pandas dataframe
    '''
    OH67 = ascii.read('OH67basesubbed/OH' + str(l) + 'p' + str(b) + '_67basesub.txt')
    OH65 = ascii.read('OH65basesubbed/OH' + str(l) + 'p' + str(b) + '_65basesub.txt')
    HI = ascii.read('HI/HI' + str(l) + 'p' + str(b) + '.txt', data_start = 13, data_end = 167)
    CO = ascii.read('CO/CO' + str(l) + 'p' + str(b) + '.txt')
    
    
    resultsTable = pd.DataFrame(np.array([[0, 
                                         0, 
                                         0,
                                         0,
                                         0,
                                         0,
                                         0,
                                         0, 
                                         0,
                                         0,
                                         0,
                                         0,
                                         0,
                                         0]]),
                               columns=['l (deg)', 
                                        'b (deg)', 
                                        'line',
                                        'comp',
                                        'T_peak (K)',
                                        'T_error (K)', 
                                        'CenterVel (km/s)',
                                        'CenterVel_error (km/s)',
                                        'FWHM (km/s)',
                                        'FWHM error (km/s)',
                                        'integral',
                                        'integral error',
                                        'Cloud',
                                        'Upper Limit'
                                       ])
    velHI = HI['col1']
    TaHI = HI['col2']
    vel67 = OH67['col1']
    Ta67 = OH67['col2']
    vel65 = OH65['col1']
    Ta65 = OH65['col2']
    velCO = CO['col1']
    TaCO = CO['col2']
    sigT67 = np.std(Ta67[np.where((-100 <= vel67) & (-25>vel67))]) #mention these in paper
    sigT65 = np.std(Ta65[np.where((-100 <= vel65) & (-25>vel65))])
    sigTCO = np.std(TaCO[np.where(velCO< -15)])
    sigTHI = np.std(TaHI[np.where(velHI< -50)])
    dVHI = velHI[1]-velHI[0] #mention these in paper
    dV67 = vel67[1]-vel67[0]
    dV65 = vel65[1]-vel65[0]
    dVCO = velCO[1]-velCO[0]    
    
    size67 = len(vel67)
    size65 = len(vel65)
    sizeHI = len(velHI)
    sizeCO = len(velCO)
    
    deltaV67 = vel67[1]-vel67[0]
    deltaV65 = vel65[1]-vel65[0]
    deltaVHI = velHI[1]-velHI[0]
    deltaVCO = velCO[1]-velCO[0]
    
    sig = 0.003 #conventional 3mK sigma

    #fit HI data
    if n_HI == 1:
        HI_p0 = np.tile(HI_p0, n_HI)
        HIlower_bounds = np.tile(HI_bounds[0], n_HI)
        HIupper_bounds = np.tile(HI_bounds[1], n_HI)
        HI_bounds = (HIlower_bounds, HIupper_bounds)
        parametersHI, covarianceHI = curve_fit(Gauss, velHI, TaHI, HI_p0, bounds = HI_bounds)

        fitHIh = [parametersHI[0]]
        fitHIc = [parametersHI[1]]
        fitHIw = [parametersHI[2]]
        perrHI = np.sqrt(np.diag(covarianceHI))
        errorHIh = [perrHI[0]]
        errorHIc = [perrHI[1]]
        errorHIw = [perrHI[2]]
                          
        v1HI = parametersHI[1] - parametersHI[2]*1.5
        v2HI = parametersHI[1] + parametersHI[2]*1.5 
                          
        intrang1 = np.where((velHI<v2HI) & (velHI>v1HI))
                          
        chan1HI = len(velHI[intrang1])
        
        #intHI_1 =  intfac * parametersHI[0] * parametersHI[2]
        intHI_1 = np.sum(TaHI[intrang1])*dVHI
        intHI = [intHI_1]
                          
        #interrHI_1 = np.sqrt(perrHI[0]**2 + perrHI[2]**2)
        interrHI_1 = np.sqrt(chan1HI)*sigTHI*dVHI
        interrHI = [interrHI_1]
        
        UpperLimHI = ['F'] #HI has no upper limits
        
        for i in range(n_HI):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        'HI',
                                                        i+1,
                                                        fitHIh[i],
                                                        errorHIh[i],
                                                        fitHIc[i],
                                                        errorHIc[i],
                                                        fitHIw[i],
                                                        errorHIw[i],
                                                        intHI[i],
                                                        interrHI[i],
                                                        Cloud,
                                                        UpperLimHI[i]
                                                        ]
    if n_HI == 2:
        
        HI_p0 = np.tile(HI_p0, n_HI)
        HIlower_bounds = np.tile(HI_bounds[0], n_HI)
        HIupper_bounds = np.tile(HI_bounds[1], n_HI)
        HI_bounds = (HIlower_bounds, HIupper_bounds)
        parametersHI, covarianceHI = curve_fit(doubleGauss, velHI, TaHI, HI_p0, bounds = HI_bounds)

        fitHIh = [parametersHI[0], parametersHI[3]]
        fitHIc = [parametersHI[1], parametersHI[4]]
        fitHIw = [parametersHI[2], parametersHI[5]]
        perrHI = np.sqrt(np.diag(covarianceHI))
        errorHIh = [perrHI[0], perrHI[3]]
        errorHIc = [perrHI[1], perrHI[4]]
        errorHIw = [perrHI[2], perrHI[5]]
        
        #first component integral range
        v1HI = parametersHI[1] - parametersHI[2]*1.5
        v2HI = parametersHI[1] + parametersHI[2]*1.5 
        #second component integral range
        v3HI = parametersHI[4] - parametersHI[5]*1.5
        v4HI = parametersHI[4] + parametersHI[5]*1.5
                          
        intrang1 = np.where((velHI<v2HI) & (velHI>v1HI))
        intrang2 = np.where((velHI<v4HI) & (velHI>v3HI))
        
        #channel number bounded
        chan1HI = len(velHI[intrang1])
        chan2HI = len(velHI[intrang2])
                          
        intHI_1 = np.sum(TaHI[intrang1])*dVHI
        intHI_2 = np.sum(TaHI[intrang2])*dVHI
        intHI = [intHI_1, intHI_2]
                          
        interrHI_1 = np.sqrt(chan1HI) * sigTHI * dVHI
        interrHI_2 = np.sqrt(chan2HI) * sigTHI * dVHI
        interrHI = [interrHI_1, interrHI_2]
        
        UpperLimHI = ['F', 'F'] #HI has no upper limits
        
        for i in range(n_HI):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        'HI',
                                                        i+1,
                                                        fitHIh[i],
                                                        errorHIh[i],
                                                        fitHIc[i],
                                                        errorHIc[i],
                                                        fitHIw[i],
                                                        errorHIw[i],
                                                        intHI[i],
                                                        interrHI[i],
                                                        Cloud,
                                                        UpperLimHI[i]
                                                        ]
            
    if n_HI == 3:
        
        HI_p0 = np.tile(HI_p0, n_HI)
        HIlower_bounds = np.tile(HI_bounds[0], n_HI)
        HIupper_bounds = np.tile(HI_bounds[1], n_HI)
        HI_bounds = (HIlower_bounds, HIupper_bounds)
        parametersHI, covarianceHI = curve_fit(tripleGauss, velHI, TaHI, HI_p0, bounds = HI_bounds)
        
        fitHIh = [parametersHI[0], parametersHI[3], parametersHI[6]]
        fitHIc = [parametersHI[1], parametersHI[4], parametersHI[7]]
        fitHIw = [parametersHI[2], parametersHI[5], parametersHI[8]]
        perrHI = np.sqrt(np.diag(covarianceHI))
        errorHIh = [perrHI[0], perrHI[3], perrHI[6]]
        errorHIc = [perrHI[1], perrHI[4], perrHI[7]]
        errorHIw = [perrHI[2], perrHI[5], perrHI[8]]
        
        #first component integral range
        v1HI = parametersHI[1] - parametersHI[2]*1.5
        v2HI = parametersHI[1] + parametersHI[2]*1.5 
        #second component integral range
        v3HI = parametersHI[4] - parametersHI[5]*1.5
        v4HI = parametersHI[4] + parametersHI[5]*1.5
        #third component integral range\
        v5HI = parametersHI[7]-parametersHI[8]*1.5
        v6HI = parametersHI[7]+parametersHI[8]*1.5
        
        #integral ranges
        intrang1 = np.where((velHI<v2HI) & (velHI>v1HI))
        intrang2 = np.where((velHI<v4HI) & (velHI>v3HI))
        intrang3 = np.where((velHI<v6HI) & (velHI>v5HI))
        
        #channel number bounded
        chan1HI = len(velHI[intrang1])
        chan2HI = len(velHI[intrang2])
        chan3HI = len(velHI[intrang3])
        
        #component integrals
        intHI_1 = np.sum(TaHI[intrang1])*dVHI
        intHI_2 = np.sum(TaHI[intrang2])*dVHI
        intHI_3 = np.sum(TaHI[intrang3])*dVHI
        intHI = [intHI_1, intHI_2, intHI_3]
        
        #component integral errors
        interrHI_1 = np.sqrt(chan1HI) * sigTHI * dVHI
        interrHI_2 = np.sqrt(chan2HI) * sigTHI * dVHI
        interrHI_3 = np.sqrt(chan3HI) * sigTHI * dVHI
        interrHI = [interrHI_1, interrHI_2, interrHI_3]
        UpperLimHI = ['F', 'F', 'F'] #HI has no upper limits

        for i in range(n_HI):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        'HI',
                                                        i+1,
                                                        fitHIh[i],
                                                        errorHIh[i],
                                                        fitHIc[i],
                                                        errorHIc[i],
                                                        fitHIw[i],
                                                        errorHIw[i],
                                                        intHI[i],
                                                        interrHI[i],
                                                        Cloud,
                                                        UpperLimHI[i]
                                                        ]
    #fit 67 data
    
    if n_67 == 1:

        OH67_p0 = np.tile(OH67_p0, n_67)
        OH67_lowerbounds = np.tile(OH67_bounds[0], n_67)
        OH67_upperbounds = np.tile(OH67_bounds[1], n_67)
        OH67_bounds = (OH67_lowerbounds, OH67_upperbounds)
        parameters67, covariance67 = curve_fit(Gauss, vel67, Ta67, OH67_p0, bounds = OH67_bounds, sigma = [sig] * size67, absolute_sigma = True)
        
        fit67h = [parameters67[0]]
        fit67c = [parameters67[1]]
        fit67w = [parameters67[2]]
        perr67 = np.sqrt(np.diag(covariance67))
        error67h = [perr67[0]]
        error67c = [perr67[1]]
        error67w = [perr67[2]]
        
        #1st comp int rang
        v167 = parameters67[1] - parameters67[2]*1.5
        v267 = parameters67[1] + parameters67[2]*1.5    
        
        #integral range
        intrang167 = np.where((vel67<v267) & (vel67>v167))
        #channels in 1st comp
        chan167 = len(vel67[intrang167])

        #integral
        int67_1 = np.sum(Ta67[intrang167])*dV67
        int67 = [int67_1]
        intuplim67 = sigT67*parameters67[2]                

        #integral error
        interr67_1 = np.sqrt(chan167)*sigT67*dV67
        interr67 = [interr67_1]
        interruplim67 = sigT67*perr67[2]
        

        
        r = n_HI - n_67
        
        
        if r == 0:
            UpperLim67 = ['F'] #1 gauss
            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            '67',
                                                            i+1,
                                                            fit67h[i],
                                                            error67h[i],
                                                            fit67c[i],
                                                            error67c[i],
                                                            fit67w[i],
                                                            error67w[i],
                                                            int67[i],
                                                            interr67[i],
                                                            Cloud,
                                                            UpperLim67[i]
                                                            ]
        elif r == 1:
            fit67h = [parameters67[0], sigT67]
            fit67c = [parameters67[1], deltaV67]
            fit67w = [parameters67[2], deltaV67]
            error67h = [perr67[0], perrHI[3]]
            error67c = [perr67[1], perrHI[4]]
            error67w = [perr67[2], perrHI[5]]
            int67 = [int67_1, intuplim67] #What do I put for upper limits for integrals?
            interr67 = [interr67_1, interruplim67]
            UpperLim67 = ['F', 'T'] #1 gauss 1 upperlim
            
            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            '67',
                                                            i+1,
                                                            fit67h[i],
                                                            error67h[i],
                                                            fit67c[i],
                                                            error67c[i],
                                                            fit67w[i],
                                                            error67w[i],
                                                            int67[i],
                                                            interr67[i],
                                                            Cloud,
                                                            UpperLim67[i]
                                                            ]       
        elif r == 2:
            fit67h = [parameters67[0], sigT67, sigT67]
            fit67c = [parameters67[1], deltaV67, deltaV67]
            fit67w = [parameters67[2], deltaV67, deltaV67]
            error67h = [perr67[0], perrHI[3], perrHI[6]]
            error67c = [perr67[1], perrHI[4], perrHI[7]]
            error67w = [perr67[2], perrHI[5], perrHI[8]]
            int67 = [int67_1, intuplim67, intuplim67]
            interr67 = [interr67_1, interruplim67, interruplim67]
            UpperLim67 = ['F', 'T', 'T'] #1 gauss 2 uplim
            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            '67',
                                                            i+1,
                                                            fit67h[i],
                                                            error67h[i],
                                                            fit67c[i],
                                                            error67c[i],
                                                            fit67w[i],
                                                            error67w[i],
                                                            int67[i],
                                                            interr67[i],
                                                            Cloud,
                                                            UpperLim67[i]
                                                            ]     
        
    if n_67 == 2:
        
        OH67_p0 = np.tile(OH67_p0, n_67)
        OH67_lowerbounds = np.tile(OH67_bounds[0], n_67)
        OH67_upperbounds = np.tile(OH67_bounds[1], n_67)
        OH67_bounds = (OH67_lowerbounds, OH67_upperbounds)
        parameters67, covariance67 = curve_fit(doubleGauss, vel67, Ta67, OH67_p0, bounds = OH67_bounds, sigma = [sig] * size67, absolute_sigma = True)

        fit67h = [parameters67[0], parameters67[3]]
        fit67c = [parameters67[1], parameters67[4]]
        fit67w = [parameters67[2], parameters67[5]]
        perr67 = np.sqrt(np.diag(covariance67))
        error67h = [perr67[0], perr67[3]]
        error67c = [perr67[1], perr67[4]]
        error67w = [perr67[2], perr67[5]]
        
        #int range by component
        v167 = parameters67[1] - parameters67[2]*1.5
        v267 = parameters67[1] + parameters67[2]*1.5    

        v367 = parameters67[4]-parameters67[5]*1.5
        v467 = parameters67[4]+parameters67[5]*1.5

        
        #integral range
        intrang167 = np.where((vel67<v267) & (vel67>v167))
        intrang267 = np.where((vel67<v467) & (vel67>v367))
        #channels per comp
        chan167 = len(vel67[intrang167])
        chan267 = len(vel67[intrang267])
    

        int67_1 = np.sum(Ta67[intrang167])*dV67
        int67_2 = np.sum(Ta67[intrang267])*dV67
        int67 = [int67_1, int67_2]
        intuplim67 = sigT67*parameters67[2]                     

        interr67_1 = np.sqrt(chan167)*sigT67*dV67
        interr67_2 = np.sqrt(chan267)*sigT67*dV67
        interr67 = [interr67_1, interr67_2]
        interruplim67 = sigT67*perr67[2]
        
        r = n_HI - n_67

        if r == 0:
            UpperLim67 = ['F', 'F'] #upper limits false here because 2 gaussian components
            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            '67',
                                                            i+1,
                                                            fit67h[i],
                                                            error67h[i],
                                                            fit67c[i],
                                                            error67c[i],
                                                            fit67w[i],
                                                            error67w[i],
                                                            int67[i],
                                                            interr67[i],
                                                            Cloud,
                                                            UpperLim67[i]
                                                            ]
                
        elif r==1:
            fit67h = [parameters67[0], parameters67[3], sigT67]
            fit67c = [parameters67[1], parameters67[4], deltaV67]
            fit67w = [parameters67[2], parameters67[5], deltaV67]
            error67h = [perr67[0],  perr67[3], perrHI[6]]
            error67c = [perr67[1],  perr67[4], perrHI[7]]
            error67w = [perr67[2],  perr67[5], perrHI[8]]
            UpperLim67 = ['F', 'F', 'T'] #2 gauss 1 uplim
            int67 = [int67_1, int67_2, intuplim67]
            interr67 = [interr67_1, interr67_2, interruplim67]
            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            '67',
                                                            i+1,
                                                            fit67h[i],
                                                            error67h[i],
                                                            fit67c[i],
                                                            error67c[i],
                                                            fit67w[i],
                                                            error67w[i],
                                                            int67[i],
                                                            interr67[i],
                                                            Cloud,
                                                            UpperLim67[i]
                                                            ]
    if n_67 == 3:
        
        OH67_p0 = np.tile(OH67_p0, n_67)
        OH67_lowerbounds = np.tile(OH67_bounds[0], n_67)
        OH67_upperbounds = np.tile(OH67_bounds[1], n_67)
        OH67_bounds = (OH67_lowerbounds, OH67_upperbounds)
        parameters67, covariance67 = curve_fit(tripleGauss, vel67, Ta67, OH67_p0, bounds = OH67_bounds, sigma = [sig] * size65, absolute_sigma = True) 
    
        fit67h = [parameters67[0], parameters67[3], parameters67[6]]
        fit67c = [parameters67[1], parameters67[4], parameters67[7]]
        fit67w = [parameters67[2], parameters67[5], parameters67[8]]
        perr67 = np.sqrt(np.diag(covariance67))
        error67h = [perr67[0], perr67[3], perr67[6]]
        error67c = [perr67[1], perr67[4], perr67[7]]
        error67w = [perr67[2], perr67[5], perr67[8]]
                         
        #int range by component, 1st number dictates component
        v167 = parameters67[1] - parameters67[2]*1.5
        v267 = parameters67[1] + parameters67[2]*1.5    

        v367 = parameters67[4]-parameters67[5]*1.5
        v467 = parameters67[4]+parameters67[5]*1.5
                         
        v567 = parameters67[7]-parameters67[8]*1.5
        v667 = parameters67[7]+parameters67[8]*1.5

        #integral range, 1st number dictates component
        intrang167 = np.where((vel67<v267) & (vel67>v167))
        intrang267 = np.where((vel67<v467) & (vel67>v367))
        intrang367 = np.where((vel67<v667) & (vel67>v567))
        #channels per comp, 1st number dictates component
        chan167 = len(vel67[intrang167])
        chan267 = len(vel67[intrang267])
        chan367 = len(vel67[intrang367])
        
        #integrals, number after underscore dictates component
        int67_1 = np.sum(Ta67[intrang167])*dV67
        int67_2 = np.sum(Ta67[intrang267])*dV67
        int67_3 = np.sum(Ta67[intrang267])*dV67
        int67 = [int67_1, int67_2, int67_3]
                          
        interr67_1 = np.sqrt(chan167)*sigT67*dV67
        interr67_2 = np.sqrt(chan267)*sigT67*dV67
        interr67_3 = np.sqrt(chan367)*sigT67*dV67
        interr67 = [interr67_1, interr67_2, interr67_3]

        for i in range(n_67):
            UpperLim67 = ['F','F','F'] #upper lim false because 3 gauss components
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        '67',
                                                        i+1,
                                                        fit67h[i],
                                                        error67h[i],
                                                        fit67c[i],
                                                        error67c[i],
                                                        fit67w[i],
                                                        error67w[i],
                                                        int67[i],
                                                        interr67[i],
                                                        Cloud,
                                                        UpperLim67[i]
                                                        ]        
    #fit 65 data
    if n_65 == 1:

        OH65_p0 = np.tile(OH65_p0, n_65)
        OH65_lowerbounds = np.tile(OH65_bounds[0], n_65)
        OH65_upperbounds = np.tile(OH65_bounds[1], n_65)
        OH65_bounds = (OH65_lowerbounds, OH65_upperbounds)        
        parameters65, covariance65 = curve_fit(Gauss, vel65, Ta65, OH65_p0, bounds = OH65_bounds, sigma = [sig] * size65, absolute_sigma = True)
        
        fit65h = [parameters65[0]]
        fit65c = [parameters65[1]]
        fit65w = [parameters65[2]]
        perr65 = np.sqrt(np.diag(covariance65))
        error65h = [perr65[0]]
        error65c = [perr65[1]]
        error65w = [perr65[2]]
                          
        #1st comp int rang
        v165 = parameters65[1] - parameters65[2]*1.5
        v265 = parameters65[1] + parameters65[2]*1.5    
        
        #integral range
        intrang165 = np.where((vel65<v265) & (vel65>v165))
        #channels in 1st comp
        chan165 = len(vel65[intrang165])

        #integral
        int65_1 = np.sum(Ta65[intrang165])*dV65
        int65 = [int65_1]
        intuplim65 = sigT65*parameters65[2]   
                          
        #integral error
        interr65_1 = np.sqrt(chan165)*sigT65*dV65
        interr65 = [interr65_1]
        interruplim65 = sigT65*perr65[2]
        
        s = n_HI - n_65
        
        if s == 0:
            UpperLim65 = ['F'] #1 gaussian comp

            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            '65',
                                                            i+1,
                                                            fit65h[i],
                                                            error65h[i],
                                                            fit65c[i],
                                                            error65c[i],
                                                            fit65w[i],
                                                            error65w[i],
                                                            int65[i],
                                                            interr65[i],
                                                            Cloud,
                                                            UpperLim65[i]
                                                            ]
        elif s == 1:
            fit65h = [parameters65[0], sigT65]
            fit65c = [parameters65[1], parameters67[1]]
            fit65w = [parameters65[2], parameters67[2]]
            error65h = [perr65[0], perrHI[3]]
            error65c = [perr65[1], perrHI[4]]
            error65w = [perr65[2], perrHI[5]]
            UpperLim65 = ['F', 'T'] #1 gauss 1 UpLim
            int65 = [int65_1, intuplim65]
            interr65 = [interr65_1, interruplim65]
            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            '65',
                                                            i+1,
                                                            fit65h[i],
                                                            error65h[i],
                                                            fit65c[i],
                                                            error65c[i],
                                                            fit65w[i],
                                                            error65w[i],
                                                            int65[i],
                                                            interr65[i],
                                                            Cloud,
                                                            UpperLim65[i]
                                                            ]       
        elif s == 2:
            fit65h = [parameters65[0], sigT65, sigT65]
            fit65c = [parameters65[1], parameters67[1], parameters67[1]]
            fit65w = [parameters65[2], parameters67[2], parameters67[2]]
            error65h = [perr65[0], perrHI[3], perrHI[6]]
            error65c = [perr65[1], perrHI[4], perrHI[7]]
            error65w = [perr65[2], perrHI[5], perrHI[8]]
            UpperLim65 = ['F', 'T', 'T'] #1 gauss 2 uplim
            int65 = [int65_1, intuplim65, intuplim65]
            interr65 = [interr65_1, interruplim65, interruplim65]
            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            '65',
                                                            i+1,
                                                            fit65h[i],
                                                            error65h[i],
                                                            fit65c[i],
                                                            error65c[i],
                                                            fit65w[i],
                                                            error65w[i],
                                                            int65[i],
                                                            interr65[i],
                                                            Cloud,
                                                            UpperLim65[i]
                                                            ]     
    if n_65 == 2:
        
        OH65_p0 = np.tile(OH65_p0, n_65)
        OH65_lowerbounds = np.tile(OH65_bounds[0], n_65)
        OH65_upperbounds = np.tile(OH65_bounds[1], n_65)
        OH65_bounds = (OH65_lowerbounds, OH65_upperbounds)  
        parameters65, covariance65 = curve_fit(doubleGauss, vel65, Ta65, OH65_p0, bounds = OH65_bounds, sigma = [sig] * size65, absolute_sigma = True)
    
        fit65h = [parameters65[0], parameters65[3]]
        fit65c = [parameters65[1], parameters65[4]]
        fit65w = [parameters65[2],parameters65[5]]
        perr65 = np.sqrt(np.diag(covariance65))
        print(perr65)
        error65h = [perr65[0], perr65[3]]
        error65c = [perr65[1], perr65[4]]
        error65w = [perr65[2], perr65[5]]
        
        #int range by component
        v165 = parameters65[1] - parameters65[2]*1.5
        v265 = parameters65[1] + parameters65[2]*1.5    

        v365 = parameters65[4]-parameters65[5]*1.5
        v465 = parameters65[4]+parameters65[5]*1.5

        #integral range
        intrang165 = np.where((vel65<v265) & (vel65>v165))
        intrang265 = np.where((vel65<v465) & (vel65>v365))
        #channels per comp
        chan165 = len(vel65[intrang165])
        chan265 = len(vel65[intrang265])
    

        int65_1 = np.sum(Ta65[intrang165])*dV65
        int65_2 = np.sum(Ta65[intrang265])*dV65
        int65 = [int65_1, int65_2]
        intuplim65 = sigT65*parameters65[2]
                          
        #interrHI_1 = np.sqrt(perrHI[0]**2 + perrHI[2]**2)
        interr65_1 = np.sqrt(chan165)*sigT65*dV65
        interr65_2 = np.sqrt(chan265)*sigT65*dV65
        interr65 = [interr65_1, interr65_2]
        interruplim65 = sigT65*perr65[2]
        
        s = n_HI - n_65
        
        if s == 0:
            UpperLim65 = ['F', 'F'] #1 gauss comp

            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            '65',
                                                            i+1,
                                                            fit65h[i],
                                                            error65h[i],
                                                            fit65c[i],
                                                            error65c[i],
                                                            fit65w[i],
                                                            error65w[i],
                                                            int65[i],
                                                            interr65[i],
                                                            Cloud,
                                                            UpperLim65[i]
                                                            ]
        elif s == 1:
            fit65h = [parameters65[0], parameters65[3], sigT65]
            fit65c = [parameters65[1], parameters65[4], parameters67[1]]
            fit65w = [parameters65[2], parameters65[4], parameters67[2]]
            error65h = [perr65[0], perr65[3], perrHI[6]]
            error65c = [perr65[1], perr65[4], perrHI[7]]
            error65w = [perr65[2], perr65[5], perrHI[8]]
            UpperLim65 = ['F', 'F', 'T'] #2 gauss 1 uplim
            int65 = [int65_1, int65_2, intuplim65]
            interr65 = [interr65_1, interr65_2, intuplim65]
            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            '65',
                                                            i+1,
                                                            fit65h[i],
                                                            error65h[i],
                                                            fit65c[i],
                                                            error65c[i],
                                                            fit65w[i],
                                                            error65w[i],
                                                            int65[i],
                                                            interr65[i],
                                                            Cloud,
                                                            UpperLim65[i]
                                                            ]       
    if n_65 == 3:
        
        OH65_p0 = np.tile(OH65_p0, n_65)
        OH65_lowerbounds = np.tile(OH65_bounds[0], n_65)
        OH65_upperbounds = np.tile(OH65_bounds[1], n_65)
        OH65_bounds = (OH65_lowerbounds, OH65_upperbounds)  
        parameters65, covariance65 = curve_fit(tripleGauss, vel65, Ta65, OH65_p0, bounds = OH65_bounds, sigma = [sig] * size65, absolute_sigma = True)    
        
        fit65h = [parameters65[0], parameters65[3], parameters65[6]]
        fit65c = [parameters65[1], parameters65[4], parameters65[7]]
        fit65w = [parameters65[2], parameters65[5], parameters65[8]]
        perr65 = np.sqrt(np.diag(covariance65))
        error65h = [perr65[0], perr65[3], perr65[6]]
        error65c = [perr65[1], perr65[4], perr65[7]]
        error65w = [perr65[2], perr65[5], perr65[8]]

        #int range by component, 1st number dictates component
        v165 = parameters65[1] - parameters65[2]*1.5
        v265 = parameters65[1] + parameters65[2]*1.5    

        v365 = parameters65[4]-parameters65[5]*1.5
        v465 = parameters65[4]+parameters65[5]*1.5
                         
        v565 = parameters65[7]-parameters65[8]*1.5
        v665 = parameters65[7]+parameters65[8]*1.5
                         
        #integral range, 1st number dictates component
        intrang165 = np.where((vel65<v265) & (vel65>v165))
        intrang265 = np.where((vel65<v465) & (vel65>v365))
        intrang365 = np.where((vel65<v665) & (vel65>v565))
        #channels per comp, 1st number dictates component
        chan165 = len(vel65[intrang165])
        chan265 = len(vel65[intrang265])
        chan365 = len(vel65[intrang365])
        
        #integrals, number after underscore dictates component
        int65_1 = np.sum(Ta65[intrang165])*dV65
        int65_2 = np.sum(Ta65[intrang265])*dV65
        int65_3 = np.sum(Ta65[intrang265])*dV65
        int65 = [int65_1, int65_2, int65_3]
                          
        #interrHI_1 = np.sqrt(perrHI[0]**2 + perrHI[2]**2)
        interr65_1 = np.sqrt(chan165)*sigT65*dV65
        interr65_2 = np.sqrt(chan265)*sigT65*dV65
        interr65_3 = np.sqrt(chan365)*sigT65*dV65
        interr65 = [interr65_1, interr65_2, interr65_3]

        
        for i in range(n_65):
            UpperLim65 = ['F', 'F', 'F'] #3 gauss comps
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        '65',
                                                        i+1,
                                                        fit65h[i],
                                                        error65h[i],
                                                        fit65c[i],
                                                        error65c[i],
                                                        fit65w[i],
                                                        error65w[i],
                                                        int65[i],
                                                        interr65[i],
                                                        Cloud,
                                                        UpperLim65[i]
                                                        ]
    #fit CO data
    if n_CO == 1:

        CO_p0 = np.tile(CO_p0, n_CO)
        CO_lowerbounds = np.tile(CO_bounds[0], n_CO)
        CO_upperbounds = np.tile(CO_bounds[1], n_CO)
        CO_bounds = (CO_lowerbounds, CO_upperbounds)  
        parametersCO, covarianceCO = curve_fit(Gauss, velCO, TaCO, CO_p0,bounds =  CO_bounds)
            
        fitCOh = [parametersCO[0]]
        fitCOc = [parametersCO[1]]
        fitCOw = [parametersCO[2]]
        perrCO = np.sqrt(np.diag(covarianceCO))
        errorCOh = [perrCO[0]]
        errorCOc = [perrCO[1]]
        errorCOw = [perrCO[2]]
                         
        #1st comp int rang
        v1CO = parametersCO[1] - parametersCO[2]*1.5
        v2CO = parametersCO[1] + parametersCO[2]*1.5    
        
        #integral range
        intrang1CO = np.where((velCO<v2CO) & (velCO>v1CO))
        #channels in 1st comp
        chan1CO = len(velCO[intrang1CO])

        #integral
        intCO_1 = np.sum(TaCO[intrang1CO])*dVCO
        intCO = [intCO_1]
        intuplimCO = sigTCO*parametersCO[2]
                          
        #integral error
        interrCO_1 = np.sqrt(chan1CO)*sigTCO*dVCO
        interrCO = [interrCO_1]
        #interrCO = [sigTCO * 5]
        interruplimCO = sigTCO*perrCO[2]
        #interruplimCO = sigTCO*5
   
        q = n_HI - n_CO
        
        if q == 0:
            UpperLimCO = ['F'] #1gauss comp

            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            'CO',
                                                            i+1,
                                                            fitCOh[i],
                                                            errorCOh[i],
                                                            fitCOc[i],
                                                            errorCOc[i],
                                                            fitCOw[i],
                                                            errorCOw[i],
                                                            intCO[i],
                                                            interrCO[i],
                                                            Cloud,
                                                            UpperLimCO[i]
                                                            ]
        elif q == 1:
            fitCOh = [parametersCO[0], sigTCO]
            fitCOc = [parametersCO[1], parameters65[1]]
            fitCOw = [parametersCO[2], parameters65[2]]
            errorCOh = [perrCO[0], perrHI[3]]
            errorCOc = [perrCO[1], perrHI[4]]
            errorCOw = [perrCO[2], perrHI[5]]
            UpperLimCO = ['F', 'T'] #1 gauss, 1 uplim
            intCO = [intCO_1, intuplimCO]
            interrCO = [interrCO_1, interruplimCO]
            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            'CO',
                                                            i+1,
                                                            fitCOh[i],
                                                            errorCOh[i],
                                                            fitCOc[i],
                                                            errorCOc[i],
                                                            fitCOw[i],
                                                            errorCOw[i],
                                                            intCO[i],
                                                            interrCO[i],
                                                            Cloud,
                                                            UpperLimCO[i]
                                                            ]       
        elif q == 2:
            fitCOh = [parametersCO[0], sigTCO, sigTCO]
            fitCOc = [parametersCO[1], parameters65[1], parameters65[1]]
            fitCOw = [parametersCO[2], parameters65[2], parameters65[2]]
            errorCOh = [perrCO[0], perrHI[3], perrHI[6]]
            errorCOc = [perrCO[1], perrHI[4], perrHI[7]]
            errorCOw = [perrCO[2], perrHI[5], perrHI[8]]
            UpperLimCO = ['F', 'T', 'T'] #1 gauss 2 uplim
            intCO = [intCO_1, intuplimCO, intuplimCO]
            interrCO = [interrCO_1, interruplimCO, interruplimCO]
            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            'CO',
                                                            i+1,
                                                            fitCOh[i],
                                                            errorCOh[i],
                                                            fitCOc[i],
                                                            errorCOc[i],
                                                            fitCOw[i],
                                                            errorCOw[i],
                                                            intCO[i],
                                                            interrCO[i],
                                                            Cloud,
                                                            UpperLimCO[i]
                                                            ]    
    if n_CO == 2:
        
        CO_p0 = np.tile(CO_p0, n_CO)
        CO_lowerbounds = np.tile(CO_bounds[0], n_CO)
        CO_upperbounds = np.tile(CO_bounds[1], n_CO)
        CO_bounds = (CO_lowerbounds, CO_upperbounds)  
        parametersCO, covarianceCO = curve_fit(doubleGauss, velCO, TaCO, CO_p0, bounds = CO_bounds)
    
        fitCOh = [parametersCO[0], parametersCO[3]]
        fitCOc = [parametersCO[1], parametersCO[4]]
        fitCOw = [parametersCO[2], parametersCO[5]] 
        perrCO = np.sqrt(np.diag(covarianceCO))
        errorCOh = [perrCO[0], perrCO[3]]
        errorCOc = [perrCO[1], perrCO[4]]
        errorCOw = [perrCO[2], perrCO[5]]
        
        #int range by component
        v1CO = parametersCO[1] - parametersCO[2]*1.5
        v2CO = parametersCO[1] + parametersCO[2]*1.5    

        v3CO = parametersCO[4]-parametersCO[5]*1.5
        v4CO = parametersCO[4]+parametersCO[5]*1.5

        #integral range
        intrang1CO = np.where((velCO<v2CO) & (velCO>v1CO))
        intrang2CO = np.where((velCO<v4CO) & (velCO>v3CO))
        #channels per comp
        chan1CO = len(velCO[intrang1CO])
        chan2CO = len(velCO[intrang2CO])
    

        intCO_1 = np.sum(TaCO[intrang1CO])*dVCO
        intCO_2 = np.sum(TaCO[intrang2CO])*dVCO
        intCO = [intCO_1, intCO_2]
        intuplimCO = sigTCO * parametersCO[2]
                          
        #interrHI_1 = np.sqrt(perrHI[0]**2 + perrHI[2]**2)
        interrCO_1 = np.sqrt(chan1CO)*sigTCO*dVCO
        interrCO_2 = np.sqrt(chan2CO)*sigTCO*dVCO
        interrCO = [interrCO_1, interrCO_2]
        interruplimCO = sigTCO*perrCO[2]
        #interruplimCO = sigTCO*5
        
        q = n_HI - n_CO
        
        if q == 0:
            UpperLimCO = ['F', 'F'] #2 gauss

            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            'CO',
                                                            i+1,
                                                            fitCOh[i],
                                                            errorCOh[i],
                                                            fitCOc[i],
                                                            errorCOc[i],
                                                            fitCOw[i],
                                                            errorCOw[i],
                                                            intCO[i],
                                                            interrCO[i],
                                                            Cloud,
                                                            UpperLimCO[i]
                                                            ]
        elif q == 1:
            fitCOh = [parametersCO[0], parametersCO[3], sigTCO]
            fitCOc = [parametersCO[1], parametersCO[4], parameters65[1]]
            fitCOw = [parametersCO[2], parametersCO[5], parameters65[2]]
            errorCOh = [perrCO[0], perrCO[3], perrHI[6]]
            errorCOc = [perrCO[1], perrCO[4], perrHI[7]]
            errorCOw = [perrCO[2], perrCO[5], perrHI[8]]
            UpperLimCO = ['F', 'F', 'T'] # 2 gauss 1 uplim
            intCO = [intCO_1, intCO_2, intuplimCO]
            interrCO = [interrCO_1, interrCO_1, interruplimCO]
            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            'CO',
                                                            i+1,
                                                            fitCOh[i],
                                                            errorCOh[i],
                                                            fitCOc[i],
                                                            errorCOc[i],
                                                            fitCOw[i],
                                                            errorCOw[i],
                                                            intCO[i],
                                                            interrCO[i],
                                                            Cloud,
                                                            UpperLimCO[i]
                                                            ]     
    if n_CO == 3:
        
        CO_p0 = np.tile(CO_p0, n_CO)
        CO_lowerbounds = np.tile(CO_bounds[0], n_CO)
        CO_upperbounds = np.tile(CO_bounds[1], n_CO)
        CO_bounds = (CO_lowerbounds, CO_upperbounds)  
        parametersCO, covarianceCO = curve_fit(tripleGauss, velCO, TaCO, CO_p0, bounds = CO_bounds) 
            
        fitCOh = [parametersCO[0], parametersCO[3], parametersCO[6]]
        fitCOc = [parametersCO[1], parametersCO[4], parametersCO[7]]
        fitCOw = [parametersCO[2], parametersCO[5], parametersCO[8]] 
        perrCO = np.sqrt(np.diag(covarianceCO))
        errorCOh = [perrCO[0], perrCO[3], perrCO[6]]
        errorCOc = [perrCO[1], perrCO[4], perrCO[7]]
        errorCOw = [perrCO[2], perrCO[5], perrCO[8]]
                         
        #int range by component, 1st number dictates component
        v1CO = parametersCO[1] - parametersCO[2]*1.5
        v2CO = parametersCO[1] + parametersCO[2]*1.5    

        v3CO = parametersCO[4]-parametersCO[5]*1.5
        v4CO = parametersCO[4]+parametersCO[5]*1.5
                         
        v5CO = parametersCO[7]-parametersCO[8]*1.5
        v6CO = parametersCO[7]+parametersCO[8]*1.5
                         
        #integral range, 1st number dictates component
        intrang1CO = np.where((velCO<v2CO) & (velCO>v1CO))
        intrang2CO = np.where((velCO<v4CO) & (velCO>v3CO))
        intrang3CO = np.where((velCO<v6CO) & (velCO>v5CO))
        #channels per comp, 1st number dictates component
        chan1CO = len(velCO[intrang1CO])
        chan2CO = len(velCO[intrang2CO])
        chan3CO = len(velCO[intrang3CO])
        
        #integrals, number after underscore dictates component
        intCO_1 = np.sum(TaCO[intrang1CO])*dVCO
        intCO_2 = np.sum(TaCO[intrang2CO])*dVCO
        intCO_3 = np.sum(TaCO[intrang2CO])*dVCO
        intCO = [intCO_1, intCO_2, intCO_3]
                          
        #interrHI_1 = np.sqrt(perrHI[0]**2 + perrHI[2]**2)
        interrCO_1 = np.sqrt(chan1CO)*sigTCO*dVCO
        interrCO_2 = np.sqrt(chan2CO)*sigTCO*dVCO
        interrCO_3 = np.sqrt(chan3CO)*sigTCO*dVCO
        interrCO = [interrCO_1, interrCO_2, interrCO_3]
        
        for i in range(n_CO):
            UpperLimCO = ['F', 'F', 'F'] #3 gauss
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        'CO',
                                                        i+1,
                                                        fitCOh[i],
                                                        errorCOh[i],
                                                        fitCOc[i],
                                                        errorCOc[i],
                                                        fitCOw[i],
                                                        errorCOw[i],
                                                        intCO[i],
                                                        interrCO[i],
                                                        Cloud,
                                                        UpperLimCO[i]
                                                        ] 
    resultsTable = resultsTable.drop(0)
    print(sigTCO)
    return resultsTable

In [4]:
df.to_csv("fit_coord_saved_dfs/119070p2800_fitted.csv", index=False)

NameError: name 'df' is not defined

In [14]:
fit_coord(119070, 2800, n_HI = 2, n_67 = 1, n_65 = 1, n_CO = 1, HI_p0 = (20, 10, 15), HI_bounds = ([0, -20, 1], [30, 30, 30]), Cloud = 'Polaris')

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),integral,integral error,Cloud,Upper Limit
1,119070,2800,HI,1,25.108944,0.262239,-4.629767,0.021005,1.847756,0.024123,128.081920,0.290790,Polaris,F
2,119070,2800,HI,2,6.805908,0.134199,-9.986011,0.218256,11.427904,0.200721,281.405440,0.755494,Polaris,F
3,119070,2800,67,1,0.086602,0.002528,-4.358241,0.037192,1.103239,0.037201,0.215733,0.004011,Polaris,F
4,119070,2800,67,2,0.002166,0.134199,0.925883,0.218256,0.925883,0.200721,0.002390,0.000081,Polaris,T
5,119070,2800,65,1,0.043520,0.002556,-4.251790,0.073304,1.081164,0.073336,0.106663,0.004928,Polaris,F
6,119070,2800,65,2,0.002658,0.134199,-4.358241,0.218256,1.103239,0.200721,0.002874,0.000195,Polaris,T
7,119070,2800,CO,1,1.613065,0.125558,-3.779779,0.077823,0.865799,0.077823,2.944761,0.191762,Polaris,F
8,119070,2800,CO,2,0.147463,0.134199,-4.251790,0.218256,1.081164,0.200721,0.127674,0.011476,Polaris,T


In [15]:
#fit_coord(119070, 2800, n_HI = 2)
df = fit_coord(119070, 2800, n_HI = 2, n_67 = 1, n_65 = 1, n_CO = 1, HI_p0 = (20, 10, 15), HI_bounds = ([0, -20, 1], [30, 30, 30]), Cloud = 'Polaris')
df.to_csv("fit_coord_saved_dfs/119070p2800_fitted.csv", index=False)
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),integral,integral error,Cloud,Upper Limit
1,119070,2800,HI,1,25.108944,0.262239,-4.629767,0.021005,1.847756,0.024123,128.081920,0.290790,Polaris,F
2,119070,2800,HI,2,6.805908,0.134199,-9.986011,0.218256,11.427904,0.200721,281.405440,0.755494,Polaris,F
3,119070,2800,67,1,0.086602,0.002528,-4.358241,0.037192,1.103239,0.037201,0.215733,0.004011,Polaris,F
4,119070,2800,67,2,0.002166,0.134199,0.925883,0.218256,0.925883,0.200721,0.002390,0.000081,Polaris,T
5,119070,2800,65,1,0.043520,0.002556,-4.251790,0.073304,1.081164,0.073336,0.106663,0.004928,Polaris,F
6,119070,2800,65,2,0.002658,0.134199,-4.358241,0.218256,1.103239,0.200721,0.002874,0.000195,Polaris,T
7,119070,2800,CO,1,1.613065,0.125558,-3.779779,0.077823,0.865799,0.077823,2.944761,0.191762,Polaris,F
8,119070,2800,CO,2,0.147463,0.134199,-4.251790,0.218256,1.081164,0.200721,0.127674,0.011476,Polaris,T


In [16]:
df = fit_coord(119210, 2000, n_HI = 3, n_67 = 2, n_65 = 1, n_CO = 1, OH67_p0 = (1, -1, 9), HI_p0 = (1, -8, 30),
          OH67_bounds = ([0, -20, 0], [3, 4, 10]), 
            HI_bounds = ([0, -25, 0], [80, 25, 30]), Cloud = 'Polaris')
df.to_csv("fit_coord_saved_dfs/119210p2000_fitted.csv", index = False)
df

#work on this fit a bit more

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),integral,integral error,Cloud,Upper Limit
1,119210,2000,HI,1,25.239004,0.566368,-11.563658,0.108095,2.904288,0.098246,213.086720,1.872514,Polaris,F
2,119210,2000,HI,2,21.241541,0.469158,-1.532379,0.156758,4.097697,0.165828,252.253440,2.238082,Polaris,F
3,119210,2000,HI,3,4.449132,0.216783,-23.845665,1.507358,24.665513,1.153242,631.040000,5.390016,Polaris,F
4,119210,2000,67,1,0.014840,0.001629,-2.937418,0.619626,2.897268,0.516489,0.155027,0.008963,Polaris,F
5,119210,2000,67,2,0.038055,0.004090,0.625262,0.072361,0.747978,0.098092,0.070404,0.004225,Polaris,F
6,119210,2000,67,3,0.003227,0.216783,0.925878,1.507358,0.925878,1.153242,0.009349,0.001667,Polaris,T
7,119210,2000,65,1,0.015542,0.002237,0.695469,0.234513,1.411649,0.234513,0.046983,0.005495,Polaris,F
8,119210,2000,65,2,0.002651,0.469158,-2.937418,0.156758,2.897268,0.165828,0.003742,0.000622,Polaris,T
9,119210,2000,65,3,0.002651,0.216783,-2.937418,1.507358,2.897268,1.153242,0.003742,0.000622,Polaris,T
10,119210,2000,CO,1,0.640718,0.144064,0.650001,0.190147,0.732573,0.190346,1.091302,0.167720,Polaris,F


In [17]:
#fit_coord(120500, 1863, n_HI = 3)
df = fit_coord(120500, 1863, n_HI = 3, HI_p0 = (1, 1, 1), 
          HI_bounds = ([0, -12, 1], [30, 5, 50]), Cloud = 'Polaris')
df.to_csv("fit_coord_saved_dfs/120500p1863_fitted.csv", index = False)
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),integral,integral error,Cloud,Upper Limit
1,120500,1863,HI,1,1.343231,0.231521,-12.000000,5.924517,48.601400,7.853127,634.375680,9.088538,Polaris,F
2,120500,1863,HI,2,19.625234,0.436840,-8.714232,0.253858,7.420977,0.227530,456.473600,3.525164,Polaris,F
3,120500,1863,HI,3,28.077985,0.884432,-0.972710,0.053691,1.793495,0.069727,167.738880,1.709955,Polaris,F
4,120500,1863,67,1,0.098978,0.002665,-0.888053,0.030852,0.991228,0.030783,0.206374,0.003936,Polaris,F
5,120500,1863,67,2,0.002454,0.436840,0.925877,0.253858,0.925877,0.227530,0.002433,0.000076,Polaris,T
6,120500,1863,67,3,0.002454,0.884432,0.925877,0.053691,0.925877,0.069727,0.002433,0.000076,Polaris,T
7,120500,1863,65,1,0.056665,0.002827,-0.806004,0.050877,0.882485,0.050795,0.113169,0.003925,Polaris,F
8,120500,1863,65,2,0.002445,0.436840,-0.888053,0.253858,0.991228,0.227530,0.002158,0.000124,Polaris,T
9,120500,1863,65,3,0.002445,0.884432,-0.888053,0.053691,0.991228,0.069727,0.002158,0.000124,Polaris,T
10,120500,1863,CO,1,2.971228,0.136213,-0.813725,0.036567,0.690755,0.036565,4.413615,0.129220,Polaris,F


In [18]:
df = fit_coord(120500, 2963, n_67 = 3, n_HI = 3, OH67_p0 = [1, 0, 2], CO_p0 = [1, -5, 0.5], 
           OH65_p0 = [-1, -8, 1], OH67_bounds = ([0, -10, 0], [3, 5, 5]), 
           OH65_bounds = ([-1, -25, 0], [0, 0, 5]), HI_bounds = ([0, -25, 0], [20, 25, 50]), Cloud = 'Polaris')
df.to_csv("fit_coord_saved_dfs/120500p2963_fitted.csv", index = False)
df

#look into 65 fit

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),integral,integral error,Cloud,Upper Limit
1,120500,2963,HI,1,4.760218,0.173414,-12.611565,0.567057,15.075809,0.426835,333.195520,4.007809,Polaris,F
2,120500,2963,HI,2,18.836095,0.399419,-3.128157,0.163163,3.312696,0.123870,194.745600,1.916099,Polaris,F
3,120500,2963,HI,3,7.079887,0.985841,-7.097375,0.088162,1.261749,0.157963,66.419200,1.173366,Polaris,F
4,120500,2963,67,1,0.014573,0.001616,-3.185757,0.551556,3.041890,0.444251,0.145395,0.007514,Polaris,F
5,120500,2963,67,2,0.030204,0.003895,-6.391253,0.090199,0.627039,0.106419,0.057774,0.003361,Polaris,F
6,120500,2963,67,3,0.000082,0.000000,4.995522,0.000000,0.000981,0.000000,0.057774,0.000000,Polaris,F
7,120500,2963,65,1,-0.083266,0.005709,-8.052769,0.055417,0.176147,0.019623,0.000000,0.000000,Polaris,F
8,120500,2963,65,2,0.002785,0.399419,-3.185757,0.163163,3.041890,0.123870,0.000491,0.000055,Polaris,T
9,120500,2963,65,3,0.002785,0.985841,-3.185757,0.088162,3.041890,0.157963,0.000491,0.000055,Polaris,T
10,120500,2963,CO,1,1.475331,0.128105,-6.307529,0.057041,0.563438,0.056081,1.961752,0.144690,Polaris,F


In [9]:
df = fit_coord(121070, 2175, n_65 = 2, n_67 = 2, n_CO = 2, n_HI = 2,
           OH65_p0 = (1, 0, 2),
           CO_bounds = ([0, -10, 0], [5, 1, 5]),
          HI_bounds = ([0, -15, 0], [20, 5, 30]), 
           OH67_bounds = ([0, -10, 0], [2, 4, 8]), 
           OH65_bounds = ([0, -10, 0], [2, 5, 3]), Cloud = 'Polaris')
df.to_csv("fit_coord_saved_dfs/121070p2175_fitted.csv", index = False)
df

[0.         0.         0.         0.00306498 0.03411201 0.02953891]


,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),integral,integral error,Cloud,Upper Limit
1,121070,2175,HI,1,9.199665,0.514994,-2.795872,0.134951,2.573755,0.172622,120.492800,0.591236,Polaris,F
2,121070,2175,HI,2,9.894463,0.414984,-6.992081,0.301407,9.146905,0.253434,252.314880,1.132132,Polaris,F
3,121070,2175,67,1,0.036667,0.001935,-3.012300,0.287766,2.051389,0.246617,0.279769,0.006831,Polaris,F
4,121070,2175,67,2,0.122155,0.005057,-6.000259,0.023289,0.666110,0.046174,0.199738,0.003652,Polaris,F
5,121070,2175,65,1,0.000475,0.000000,1.333681,0.000000,0.003220,0.000000,0.000000,0.000000,Polaris,F
6,121070,2175,65,2,0.077346,0.003065,-5.794757,0.034112,0.676482,0.029539,0.106927,0.003123,Polaris,F
7,121070,2175,CO,1,3.861263,0.274407,-6.122722,0.043057,0.780494,0.048677,7.105254,0.145358,Polaris,F
8,121070,2175,CO,2,1.935919,0.091977,-3.416223,0.234136,1.650386,0.196383,9.839554,0.237369,Polaris,F


In [15]:
df = fit_coord(121930, 1975, n_HI = 2, n_67 = 2, OH67_p0 = (.5, 0, 1), HI_p0 = (1.3, -10, 4),
           OH65_p0 = (1, 0, 2), 
           OH65_bounds = ([0, 0, 0], [1, 10, 5]), 
           OH67_bounds = ([0, -5, 0], [.5, 5, 1]), 
           HI_bounds = ([0, -20, 1], [40, 5, 50]), Cloud = 'Polaris') 

df.to_csv("fit_coord_saved_dfs/121930p1975_fitted.csv", index = False)
df
#high error????

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),integral,integral error,Cloud,Upper Limit
1,121930,1975,HI,1,19.510558,1.101197,-0.612561,0.303869,3.564306,0.254745,172.092160,4.882709,Polaris,F
2,121930,1975,HI,2,18.323914,0.657320,-12.521152,0.460621,5.477727,0.425025,238.424320,6.224257,Polaris,F
3,121930,1975,67,1,0.017690,233.070543,0.716072,688.794174,0.138198,2974.886891,0.019814,0.002425,Polaris,F
4,121930,1975,67,2,0.018097,360.524998,0.727192,676.080646,0.140004,2907.260462,0.019814,0.002425,Polaris,F
5,121930,1975,65,1,0.057793,732.992316,1.298861,66.457263,0.255005,979.385740,0.000000,0.000000,Polaris,F
6,121930,1975,65,2,0.002599,0.657320,0.716072,0.460621,0.138198,0.425025,0.000663,2.545733,Polaris,T
7,121930,1975,CO,1,0.859033,0.160622,1.332860,0.119604,0.569361,0.125836,0.947458,0.155391,Polaris,F
8,121930,1975,CO,2,0.168991,0.657320,1.298861,0.460621,0.255005,0.425025,0.096217,0.021265,Polaris,T


In [21]:
df = fit_coord(122220, 1875, n_HI = 3, HI_p0 = (1, -10, 25), CO_p0 = (1, 2, 0.5),
          CO_bounds = ([0, 0, 0.1], [3, 3, 3]), 
          HI_bounds = ([0, -20, 1], [30, 20, 50]), Cloud = 'Polaris')
df.to_csv("fit_coord_saved_dfs/122220p1875_fitted.csv", index = False)
df
#CHECK LATER ON CO ERROR (basically just noise as seen in plot)

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),integral,integral error,Cloud,Upper Limit
1,122220,1875,HI,1,7.717729,5.669890e+07,-8.303448,1.609865e+06,9.320453,5.227957e+05,473.415680,10.245328,Polaris,F
2,122220,1875,HI,2,7.717792,4.832427e+07,-8.302470,3.658915e+07,9.324325,3.987894e+07,473.415680,10.245328,Polaris,F
3,122220,1875,HI,3,7.717854,3.760590e+07,-8.302620,3.737444e+07,9.324274,4.040713e+07,473.415680,10.245328,Polaris,F
4,122220,1875,67,1,0.027175,2.952555e-03,1.999606,1.013715e-01,0.803392,1.004038e-01,0.049718,0.003478,Polaris,F
5,122220,1875,67,2,0.002169,4.832427e+07,0.925876,3.658915e+07,0.925876,3.987894e+07,0.001742,0.000218,Polaris,T
6,122220,1875,67,3,0.002169,3.760590e+07,0.925876,3.737444e+07,0.925876,4.040713e+07,0.001742,0.000218,Polaris,T
7,122220,1875,65,1,0.013580,2.473203e-03,1.981341,2.427524e-01,1.154046,2.427510e-01,0.032889,0.004508,Polaris,F
8,122220,1875,65,2,0.002432,4.832427e+07,1.999606,3.658915e+07,0.803392,3.987894e+07,0.002806,0.000590,Polaris,T
9,122220,1875,65,3,0.002432,3.760590e+07,1.999606,3.737444e+07,0.803392,4.040713e+07,0.002806,0.000590,Polaris,T
10,122220,1875,CO,1,2.180083,4.531807e+03,1.964676,1.972892e+01,0.192040,1.395386e+02,0.000000,0.000000,Polaris,F


In [27]:
#df = fit_coord(122500, 2900, n_HI = 3, n_67 = 3, 
           #HI_p0 = (1, -10, 25), OH67_p0 = (1, -3, 4),
           #CO_p0 = (2, -4, 3),
           #OH67_bounds = ([0, -8, 1], [5, 3, 10]), 
           #CO_bounds = ([0, -10, 1], [5, -1, 5]), HI_bounds = ([0, -30, 1], [20, -3, 30]), Cloud = 'Polaris')
#df.to_csv("fit_coord_saved_dfs/122500p2900_fitted.csv", index = False)
#df
#issue with OH bounds and CO 

In [57]:
#df = fit_coord(123360, 2075, n_HI = 3, HI_p0 = (2, -3, 1), n_65 = 2, OH65_p0 = (1, 0, 2), 
           n_67 = 3, OH67_p0 = (1, 0, 4), n_CO = 2, CO_p0 = (1, 0, 3)
          , OH67_bounds = ([0, -10, 0], [4, 5, 10]), OH65_bounds = ([0, -10, 0], [4, 5, 2]),
          CO_bounds = ([0, -5, 0], [4, 4, 5]), HI_bounds = ([0, -20, 0], [30, 20, 10]), Cloud = 'Polaris')
#df.to_csv("fit_coord_saved_dfs/123360p2075_fitted.csv", index = False)
#df

#67 temperature peak questionable

[0.00426241 0.08663692 0.14069185 0.00195452 0.46897113 0.48562294]


,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),integral,integral error,Cloud,Upper Limit
1,123360,2075,HI,1,18.089126,0.598025,-12.401719,0.341523,5.083368,0.339976,239.055360,4.016655,Polaris,F
2,123360,2075,HI,2,25.697305,1.364065,-3.592403,0.114687,1.981039,0.135200,152.012800,2.592739,Polaris,F
3,123360,2075,HI,3,19.038650,0.766518,4.564558,0.175269,3.071973,0.192284,141.478400,3.279585,Polaris,F
4,123360,2075,67,1,0.029108,0.004767,-0.259643,0.395666,0.991864,0.373811,0.068133,0.003238,Polaris,F
5,123360,2075,67,2,0.012540,0.005995,-2.629169,0.997715,0.978566,0.626821,0.034101,0.003238,Polaris,F
6,123360,2075,67,3,0.067711,0.002964,2.793985,0.059924,0.803653,0.056713,0.034101,0.003238,Polaris,F
7,123360,2075,65,1,0.034299,0.004262,2.947831,0.086637,0.622117,0.140692,0.053455,0.003325,Polaris,F
8,123360,2075,65,2,0.015391,0.001955,-0.537367,0.468971,0.537367,0.485623,0.067895,0.005758,Polaris,F
9,123360,2075,65,3,0.002536,0.766518,-0.259643,0.175269,0.991864,0.192284,0.001578,0.001578,Polaris,T
10,123360,2075,CO,1,1.623682,0.091746,-0.481580,0.086190,1.122114,0.094524,4.008662,0.181538,Polaris,F


In [37]:
df = fit_coord(123500, 1963, n_67 = 2, n_HI = 3, n_CO = 2, HI_p0 = (15, 0, 5), OH65_p0 = (2, 0, 2),
           CO_p0 = (.4, -2, .4), CO_bounds = ([0, -10, 0], [2, 0, 10]), OH67_p0 = (1, 0, 2), 
          OH65_bounds = ([0, -10, 0], [5, .3, 5]), 
           HI_bounds= ([10, -30, 1], [80, 25, 15]), OH67_bounds = ([0, -10, 0], [5, 10, 7]), Cloud = 'Polaris')

df.to_csv("fit_coord_saved_dfs/123500p1963_fitted.csv", index = False)
df
#cant get 65 to fit

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),integral,integral error,Cloud,Upper Limit
1,123500,1963,HI,1,25.697473,1.364044,-3.592408,0.114687,1.981047,0.135200,152.012800,2.592739,Polaris,F
2,123500,1963,HI,2,18.089198,0.598028,-12.401742,0.341514,5.083302,0.339970,239.055360,4.016655,Polaris,F
3,123500,1963,HI,3,19.038663,0.766517,4.564557,0.175270,3.071973,0.192284,141.478400,3.279585,Polaris,F
4,123500,1963,67,1,0.008113,0.002100,5.086275,0.478077,1.599966,0.478184,0.025929,0.004498,Polaris,F
5,123500,1963,67,2,0.034361,0.002180,-3.964759,0.108743,1.484738,0.108766,0.113258,0.004498,Polaris,F
6,123500,1963,67,3,0.002173,0.766517,0.925885,0.175270,0.925885,0.192284,0.003476,0.001039,Polaris,T
7,123500,1963,65,1,0.021746,0.002044,-3.947261,0.183431,1.690185,0.183431,0.074590,0.004216,Polaris,F
8,123500,1963,65,2,0.002034,0.598028,5.086275,0.341514,1.599966,0.339970,0.003438,0.000373,Polaris,T
9,123500,1963,65,3,0.002034,0.766517,5.086275,0.175270,1.599966,0.192284,0.003438,0.000373,Polaris,T
10,123500,1963,CO,1,0.527672,0.186162,-4.651496,0.416294,1.433705,0.298391,2.029244,0.171166,Polaris,F


In [38]:
df = fit_coord(125070, 1950, n_HI = 2, HI_p0 = (10, 1, 20), 
               HI_bounds = ([0, -30, 0], [20, 10, 30]), Cloud = 'Polaris')
df.to_csv("fit_coord_saved_dfs/125070p1950_fitted.csv", index = False)
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),integral,integral error,Cloud,Upper Limit
1,125070,1950,HI,1,16.459177,0.269666,-8.096937,0.399490,10.082063,0.284660,419.511040,2.539420,Polaris,F
2,125070,1950,HI,2,8.403189,0.718997,4.079468,0.195455,3.464642,0.308160,143.176960,1.466135,Polaris,F
3,125070,1950,67,1,0.106404,0.003160,0.660790,0.024930,0.620396,0.020450,0.139287,0.003648,Polaris,F
4,125070,1950,67,2,0.002786,0.718997,0.925884,0.195455,0.925884,0.308160,0.001729,0.000057,Polaris,T
5,125070,1950,65,1,0.059559,0.003096,0.686141,0.047755,0.555600,0.032003,0.054653,0.002007,Polaris,F
6,125070,1950,65,2,0.002165,0.718997,0.660790,0.195455,0.620396,0.308160,0.001203,0.000069,Polaris,T
7,125070,1950,CO,1,1.962675,0.147545,0.680557,0.051534,0.601743,0.052906,2.210882,0.131926,Polaris,F
8,125070,1950,CO,2,0.143473,0.718997,0.686141,0.195455,0.555600,0.308160,0.086334,0.007591,Polaris,T


In [39]:
df = fit_coord(125220, 3250, OH67_bounds = ([0, -5, 0], [3, 5, 5]), Cloud = 'Polaris')
df.to_csv("fit_coord_saved_dfs/125220p3250_fitted.csv", index = False)
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),integral,integral error,Cloud,Upper Limit
1,125220,3250,HI,1,36.662619,0.776355,1.152745,0.067798,2.775750,0.067794,225.839360,3.423336,Polaris,F
2,125220,3250,67,1,0.120393,0.002593,0.069041,0.026084,1.048728,0.026070,0.261557,0.003814,Polaris,F
3,125220,3250,65,1,0.068332,0.002713,0.234129,0.044017,0.961648,0.044164,0.139975,0.003612,Polaris,F
4,125220,3250,CO,1,0.863866,0.103696,0.389683,0.137698,0.993339,0.137698,1.873039,0.167259,Polaris,F


In [40]:
df = fit_coord(125500, 2200, n_HI = 2, n_67 = 2, n_65 = 2, OH67_p0 = (1, 0, 2)
          , OH67_bounds = ([0, -5, 0], [1.5, 5, 5]), HI_bounds = ([0, -20, 0], [20, 15, 30]),
          OH65_bounds = ([0, -5, 0], [1.5, 5, 5]), Cloud = 'Polaris') 
df.to_csv("fit_coord_saved_dfs/125500p2200_fitted.csv", index = False)
df

#note, double check this is in polaris later!!

[0.00318171 0.75151665 0.53575662 0.00453264 0.10368394 0.124889  ]


,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),integral,integral error,Cloud,Upper Limit
1,125500,2200,HI,1,6.079098,0.242600,-9.139407,0.554356,16.841553,0.536623,347.508480,1.070702,Polaris,F
2,125500,2200,HI,2,14.435099,0.377698,-2.672341,0.092363,3.392014,0.111442,161.118720,0.484933,Polaris,F
3,125500,2200,67,1,0.056400,0.004332,-4.709768,0.055014,0.739635,0.060019,0.134519,0.003347,Polaris,F
4,125500,2200,67,2,0.016239,0.003111,-3.185540,0.476808,2.474596,0.334487,0.190477,0.005465,Polaris,F
5,125500,2200,65,1,0.009469,0.003182,-3.344340,0.751517,2.205602,0.535757,0.090966,0.004863,Polaris,F
6,125500,2200,65,2,0.029500,0.004533,-4.582643,0.103684,0.594375,0.124889,0.054644,0.002599,Polaris,F
7,125500,2200,CO,1,2.437001,0.095121,-4.804518,0.038853,0.911051,0.043082,4.790790,0.072701,Polaris,F
8,125500,2200,CO,2,0.039544,0.377698,-3.344340,0.092363,2.205602,0.111442,0.036027,0.001704,Polaris,T


In [42]:
df = fit_coord(127500, 2088, n_67 = 2, n_65 = 2, 
           n_HI = 3, OH67_p0 = (4, 0, 3), 
           OH65_p0 = (2, 0, 3), 
           HI_p0 = (10, 0, 30), 
           OH65_bounds = ([0, -15, 1], [2, 10, 10]), 
           OH67_bounds = ([0, -15, 1], [15, 5, 10]),
          HI_bounds = ([0, -30, 1], [30, 18, 40]), Cloud = 'Polaris')
df.to_csv("fit_coord_saved_dfs/127500p2088_fitted.csv", index = False)
df

[0.00157978 1.03726584 1.08702151 0.00180765 0.18123505 0.18664825]


,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),integral,integral error,Cloud,Upper Limit
1,127500,2088,HI,1,16.973268,0.247301,6.196844,0.135649,4.078035,0.123679,196.199680,0.434048,Polaris,F
2,127500,2088,HI,2,11.197018,0.173359,-13.794744,0.298294,7.508598,0.269080,250.393600,0.565929,Polaris,F
3,127500,2088,HI,3,24.285442,0.601601,-2.632001,0.066571,2.669137,0.072193,185.484800,0.336212,Polaris,F
4,127500,2088,67,1,0.049094,0.002518,-1.098456,0.090182,1.737863,0.110833,0.197193,0.005240,Polaris,F
5,127500,2088,67,2,0.006635,0.001598,3.496526,1.835255,6.693870,1.283248,0.311641,0.010739,Polaris,F
6,127500,2088,67,3,0.002531,0.601601,0.925885,0.066571,0.925885,0.072193,0.004399,0.000281,Polaris,T
7,127500,2088,65,1,0.005158,0.001580,9.148051,1.037266,2.899782,1.087022,0.033424,0.006792,Polaris,F
8,127500,2088,65,2,0.025791,0.001808,-1.390440,0.181235,-1.390440,0.186648,0.122163,0.005990,Polaris,F
9,127500,2088,65,3,0.002442,0.601601,-1.098456,0.066571,1.737863,0.072193,0.007082,0.007082,Polaris,T
10,127500,2088,CO,1,2.598015,0.103059,-0.610891,0.059003,1.288129,0.059003,7.393872,0.216531,Polaris,F


In [14]:
#this is coordinate with absolutely 0 CO
df = fit_coord(137160, 4300, OH67_p0 = (1, 10, 1), n_HI = 2,
          HI_bounds = ([0, 0, 1], [20 ,15, 15]), 
          CO_bounds = ([0, -1, 0], [1, 1, 1]), Cloud = 'Ursa Major')
df.to_csv("fit_coord_saved_dfs/137160p4300_fitted.csv", index = False)
df

0.0


,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),integral,integral error,Cloud,Upper Limit
1,137160,4300,HI,1,2.592080,0.158710,3.875292e-16,0.808044,10.876015,0.658176,107.824640,2.022683,Ursa Major,F
2,137160,4300,HI,2,11.218612,0.357025,7.849035e+00,0.061405,1.756082,0.069653,58.478080,0.904571,Ursa Major,F
3,137160,4300,67,1,0.010049,0.002926,8.702887e+00,0.277082,0.823589,0.276742,0.018839,0.003422,Ursa Major,F
4,137160,4300,67,2,0.002133,0.357025,9.259306e-01,0.061405,0.925931,0.069653,0.001757,0.000590,Ursa Major,T
5,137160,4300,65,1,0.003101,0.002081,-1.008849e+00,1.263051,1.629755,1.263051,0.011930,0.004780,Ursa Major,F
6,137160,4300,65,2,0.002306,0.357025,8.702887e+00,0.061405,0.823589,0.069653,0.003759,0.002913,Ursa Major,T
7,137160,4300,CO,1,0.024321,0.000140,4.999433e-04,0.002414,0.084556,0.000595,0.000000,0.000000,Ursa Major,F
8,137160,4300,CO,2,0.000000,0.357025,-1.008849e+00,0.061405,1.629755,0.069653,0.000000,0.000000,Ursa Major,T


In [44]:
df = fit_coord(142210, 2325, n_HI = 3, HI_p0 = (1, 0, 8), 
           HI_bounds = ([0, -30, 0], [20, 5, 50]), Cloud = 'Camelopardalis')
df.to_csv("fit_coord_saved_dfs/142210p2325_fitted.csv", index = False)
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),integral,integral error,Cloud,Upper Limit
1,142210,2325,HI,1,14.290682,0.283359,-3.126089,0.110026,8.025419,0.147917,340.113920,0.661419,Camelopardalis,F
2,142210,2325,HI,2,19.395987,0.384563,-1.330761,0.033308,1.744758,0.042756,145.198080,0.311796,Camelopardalis,F
3,142210,2325,HI,3,0.818782,0.095787,-28.850741,5.007794,36.470075,3.877941,435.892480,1.428828,Camelopardalis,F
4,142210,2325,67,1,0.084863,0.002897,-2.042813,0.032881,0.820334,0.031870,0.158468,0.004134,Camelopardalis,F
5,142210,2325,67,2,0.002578,0.384563,0.925878,0.033308,0.925878,0.042756,0.002115,0.000082,Camelopardalis,T
6,142210,2325,67,3,0.002578,0.095787,0.925878,5.007794,0.925878,3.877941,0.002115,0.000082,Camelopardalis,T
7,142210,2325,65,1,0.043649,0.002917,-1.955868,0.063117,0.789529,0.059132,0.079873,0.004105,Camelopardalis,F
8,142210,2325,65,2,0.002557,0.384563,-2.042813,0.033308,0.820334,0.042756,0.002019,0.000151,Camelopardalis,T
9,142210,2325,65,3,0.002557,0.095787,-2.042813,5.007794,0.820334,3.877941,0.002019,0.000151,Camelopardalis,T
10,142210,2325,CO,1,2.504879,0.131268,-2.169310,0.043053,0.711052,0.043011,3.731103,0.170954,Camelopardalis,F


In [45]:
df = fit_coord(143000, 3850, n_HI = 2, HI_bounds = ([0, -10, 0], [20, 15, 30]), Cloud = 'Ursa Major')
df.to_csv("fit_coord_saved_dfs/143000p3850_fitted.csv", index = False)
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),integral,integral error,Cloud,Upper Limit
1,143000,3850,HI,1,17.597761,1.611605,4.112793,0.276430,3.666825,0.175857,185.249280,6.134445,Ursa Major,F
2,143000,3850,HI,2,12.805716,1.821154,2.037580,0.150874,1.359356,0.202025,114.204160,4.089630,Ursa Major,F
3,143000,3850,67,1,0.097741,0.002780,2.225671,0.030040,0.918526,0.030299,0.191408,0.003860,Ursa Major,F
4,143000,3850,67,2,0.002407,1.821154,0.925871,0.150874,0.925871,0.202025,0.002211,0.000073,Ursa Major,T
5,143000,3850,65,1,0.053961,0.002779,2.292664,0.054513,0.921137,0.055060,0.103609,0.003634,Ursa Major,F
6,143000,3850,65,2,0.002264,1.821154,2.225671,0.150874,0.918526,0.202025,0.002085,0.000125,Ursa Major,T
7,143000,3850,CO,1,2.491926,0.101556,2.302071,0.037419,0.795076,0.037414,3.959732,0.133531,Ursa Major,F
8,143000,3850,CO,2,0.118570,1.821154,2.292664,0.150874,0.921137,0.202025,0.094272,0.004436,Ursa Major,T


In [46]:
df = fit_coord(146070, 1775, n_HI = 2, OH67_bounds = ([0, -5, 0], [20, 5, 5])
          , OH65_bounds = ([0, -5, 0], [20, 5, 5]), HI_bounds = ([0, -20, 0], [20, 8, 30]), Cloud = 'Camelopardalis')
df.to_csv("fit_coord_saved_dfs/146070p1775_fitted.csv", index = False)
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),integral,integral error,Cloud,Upper Limit
1,146070,1775,HI,1,20.000000,0.976250,0.743232,0.078731,1.976825,0.105157,161.244160,0.845951,Camelopardalis,F
2,146070,1775,HI,2,17.389602,0.882943,-1.931719,0.210940,5.902985,0.168968,317.667840,1.582629,Camelopardalis,F
3,146070,1775,67,1,0.230267,0.003001,0.487612,0.011596,0.723211,0.010439,0.398132,0.003972,Camelopardalis,F
4,146070,1775,67,2,0.002477,0.882943,0.925905,0.210940,0.925905,0.168968,0.001791,0.000026,Camelopardalis,T
5,146070,1775,65,1,0.169170,0.002986,0.591286,0.015728,0.687076,0.013057,0.282429,0.003599,Camelopardalis,F
6,146070,1775,65,2,0.002242,0.882943,0.487612,0.210940,0.723211,0.168968,0.001540,0.000029,Camelopardalis,T
7,146070,1775,CO,1,4.350812,0.114572,0.258126,0.027344,0.899220,0.027344,8.407118,0.181272,Camelopardalis,F
8,146070,1775,CO,2,0.139399,0.882943,0.591286,0.210940,0.687076,0.168968,0.125351,0.003812,Camelopardalis,T


In [47]:
df = fit_coord(146200, 3963, n_HI = 2, HI_p0 = (1, 0, 5), HI_bounds = ([0, -10, 0], [20, 15, 30]), Cloud = 'Ursa Major')
df.to_csv("fit_coord_saved_dfs/146200p3963_fitted.csv", index = False)
df

#weird HI bounds behavior when varying upper bounds

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),integral,integral error,Cloud,Upper Limit
1,146200,3963,HI,1,9.588239,0.809138,-2.501411,0.378130,3.087515,0.413427,65.146880,6.883982,Ursa Major,F
2,146200,3963,HI,2,15.570930,1.043691,5.265048,0.177999,1.844501,0.173671,70.961920,5.818027,Ursa Major,F
3,146200,3963,67,1,0.022717,0.002383,-0.098259,0.150440,1.241909,0.150438,0.060795,0.004390,Ursa Major,F
4,146200,3963,67,2,0.002371,1.043691,0.925878,0.177999,0.925878,0.173671,0.002944,0.000357,Ursa Major,T
5,146200,3963,65,1,0.017295,0.003539,0.399105,0.142963,0.695627,0.177370,0.025646,0.002951,Ursa Major,F
6,146200,3963,65,2,0.002251,1.043691,-0.098259,0.177999,1.241909,0.173671,0.001566,0.000399,Ursa Major,T
7,146200,3963,CO,1,1.779890,0.092886,0.001515,0.049196,0.816434,0.049200,3.257466,0.137518,Ursa Major,F
8,146200,3963,CO,2,0.105750,1.043691,0.399105,0.177999,0.695627,0.173671,0.086338,0.005203,Ursa Major,T


In [48]:
df = fit_coord(146930, 2075, Cloud = 'Camelopardalis')
df.to_csv("fit_coord_saved_dfs/146930p2075_fitted.csv", index = False)
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),integral,integral error,Cloud,Upper Limit
1,146930,2075,HI,1,37.110928,0.772866,-1.210916,0.072475,3.014247,0.072489,234.173440,0.753896,Camelopardalis,F
2,146930,2075,67,1,0.057882,0.002881,-0.134523,0.048703,0.841744,0.048108,0.111347,0.009412,Camelopardalis,F
3,146930,2075,65,1,0.033501,0.004148,0.136408,0.068494,0.642267,0.102316,0.047874,0.008733,Camelopardalis,F
4,146930,2075,CO,1,0.697665,0.092378,0.055885,0.106260,0.695648,0.106522,1.136353,0.138728,Camelopardalis,F


In [49]:
df = fit_coord(147000, 4038, n_CO = 2, n_67 = 2, n_HI = 2, n_65 = 2, OH67_bounds = ([0, -15, 0], [1, 3, 5])
          , OH65_bounds = ([0, -5, 0], [1, 3, 5]), CO_bounds = ([0, -8, 0], [2, 5, 5]), HI_bounds = ([0, -50, 0], [20, 15, 50]), Cloud = 'Ursa Major')
df.to_csv("fit_coord_saved_dfs/147000p4038_fitted.csv", index = False)
df


[0.00409903 0.16810189 0.21057839 0.00273502 0.64768404 0.80591135]


,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),integral,integral error,Cloud,Upper Limit
1,147000,4038,HI,1,0.875853,0.064871,-34.594984,3.398961,32.054907,3.135367,189.368320,5.351557,Ursa Major,F
2,147000,4038,HI,2,11.054588,0.177355,0.713807,0.081339,4.668746,0.093314,114.065920,1.954112,Ursa Major,F
3,147000,4038,67,1,0.007669,0.002783,1.023233,0.572745,0.931524,0.568057,0.023010,0.004505,Ursa Major,F
4,147000,4038,67,2,0.041632,0.002765,-1.883539,0.105968,0.942039,0.105078,0.087538,0.004505,Ursa Major,F
5,147000,4038,65,1,0.016685,0.004099,-1.267346,0.168102,0.593682,0.210578,0.038139,0.003197,Ursa Major,F
6,147000,4038,65,2,0.008832,0.002735,-0.919004,0.647684,3.258312,0.805911,0.090537,0.007497,Ursa Major,F
7,147000,4038,CO,1,0.607109,0.128378,1.116380,0.126586,0.495946,0.123473,0.624182,0.108020,Ursa Major,F
8,147000,4038,CO,2,1.542829,0.100168,-1.863682,0.063116,0.841034,0.064972,2.886843,0.152763,Ursa Major,F


In [50]:
df = fit_coord(147200, 4075, n_67 = 2, n_65 = 1, n_CO = 2, n_HI = 2,
           OH65_bounds = ([0, -15, 0], [2, 8, 5]),
           OH67_bounds = ([0, -15, 0], [2, 8, 5]),
          CO_bounds = ([0, 0, 0], [3, 10, 2]), HI_bounds = ([0, -15, 0], [50, 30, 50]), Cloud = 'Ursa Major')  
df.to_csv("fit_coord_saved_dfs/147200p4075_fitted.csv", index = False)
df

#issue when I apply bounds to 65

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),integral,integral error,Cloud,Upper Limit
1,147200,4075,HI,1,6.923539,0.622044,3.855904,0.095112,1.031301,0.103483,68.110080,0.888177,Ursa Major,F
2,147200,4075,HI,2,17.243302,0.393698,1.226618,0.100444,3.903025,0.071469,162.384640,1.538368,Ursa Major,F
3,147200,4075,67,1,0.025082,0.002510,-1.748482,0.129354,1.119463,0.129376,0.056244,0.003732,Ursa Major,F
4,147200,4075,67,2,0.076728,0.002969,3.683673,0.034997,0.731117,0.031112,0.134257,0.003732,Ursa Major,F
5,147200,4075,65,1,0.054946,0.003199,3.812877,0.047296,0.683667,0.045846,0.078358,0.002733,Ursa Major,F
6,147200,4075,65,2,0.002084,0.393698,-1.748482,0.100444,1.119463,0.071469,0.001425,0.000096,Ursa Major,T
7,147200,4075,CO,1,1.999997,54.032153,3.668533,1.479013,0.603584,1.224599,5.695725,0.135045,Ursa Major,F
8,147200,4075,CO,2,1.999997,54.010936,3.789744,1.938420,0.689135,1.124870,5.695725,0.135045,Ursa Major,F


In [51]:
df = fit_coord(148200, 3863, n_HI = 2, HI_p0 = (1, 0, 5), 
           HI_bounds = ([0, -10, 1], [20, 10, 20]), Cloud = 'Ursa Major')
df.to_csv("fit_coord_saved_dfs/148200p3863_fitted.csv", index = False)
df


,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),integral,integral error,Cloud,Upper Limit
1,148200,3863,HI,1,8.934364,1.033569,1.100540,0.461010,5.912687,0.459924,143.765760,7.664408,Ursa Major,F
2,148200,3863,HI,2,7.813048,1.254321,-0.932991,0.231442,1.572277,0.301963,70.949120,4.096799,Ursa Major,F
3,148200,3863,67,1,0.047720,0.002417,-1.827805,0.070603,1.207130,0.070602,0.129298,0.004606,Ursa Major,F
4,148200,3863,67,2,0.002488,1.254321,0.925884,0.231442,0.925884,0.301963,0.003003,0.000176,Ursa Major,T
5,148200,3863,65,1,0.041634,0.002856,-2.200168,0.068436,0.860289,0.067855,0.080700,0.004024,Ursa Major,F
6,148200,3863,65,2,0.002506,1.254321,-1.827805,0.231442,1.207130,0.301963,0.002156,0.000170,Ursa Major,T
7,148200,3863,CO,1,1.576454,0.102998,-2.316782,0.052379,0.693315,0.052232,2.301702,0.125272,Ursa Major,F
8,148200,3863,CO,2,0.111236,1.254321,-2.200168,0.231442,0.860289,0.301963,0.077122,0.005810,Ursa Major,T


In [52]:
df = fit_coord(148210, 2350, n_HI = 2, HI_p0 = (1, 0, 5), HI_bounds = ([0, -20, 0], [20, 15, 30]),
           OH67_p0 = (1, 1, 1),
          OH67_bounds = ([0, 0, 0], [1, 5, 2]), Cloud = 'Camelopardalis')
df.to_csv("fit_coord_saved_dfs/148210p2350_fitted.csv", index = False)
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),integral,integral error,Cloud,Upper Limit
1,148210,2350,HI,1,10.816930,0.469055,1.905394,0.061185,1.293293,0.070372,71.696640,0.564118,Camelopardalis,F
2,148210,2350,HI,2,11.950760,0.233022,-1.813389,0.157706,8.475088,0.148829,253.838080,1.456546,Camelopardalis,F
3,148210,2350,67,1,0.066329,0.003094,1.370475,0.039619,0.671339,0.034762,0.091060,0.003398,Camelopardalis,F
4,148210,2350,67,2,0.002595,0.233022,0.925906,0.157706,0.925906,0.148829,0.001742,0.000090,Camelopardalis,T
5,148210,2350,65,1,0.044365,0.003422,1.283639,0.059285,0.609142,0.054886,0.059264,0.003994,Camelopardalis,F
6,148210,2350,65,2,0.003047,0.233022,1.370475,0.157706,0.671339,0.148829,0.001856,0.000167,Camelopardalis,T
7,148210,2350,CO,1,3.165647,0.141303,1.304324,0.026811,0.545670,0.029165,3.444943,0.121024,Camelopardalis,F
8,148210,2350,CO,2,0.131618,0.233022,1.283639,0.157706,0.609142,0.148829,0.071820,0.003839,Camelopardalis,T


In [13]:
df = fit_coord(150120, 4067, CO_bounds = ([0, -1, 0], [1, 1, 1]), Cloud = 'Ursa Major')
df.to_csv("fit_coord_saved_dfs/150120p4067_fitted.csv", index = False)
df

0.0


,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),integral,integral error,Cloud,Upper Limit
1,150120,4067,HI,1,10.608966,0.261963,1.221581,0.148003,5.192746,0.148060,118.141440,3.256637,Ursa Major,F
2,150120,4067,67,1,0.003397,0.001488,-3.658921,1.611335,3.183563,1.611335,0.023558,0.006776,Ursa Major,F
3,150120,4067,65,1,0.002466,0.001126,1.338215,2.935009,5.567352,2.935009,0.037056,0.009872,Ursa Major,F
4,150120,4067,CO,1,0.024321,0.000140,0.000500,0.002414,0.084556,0.000595,0.000000,0.000000,Ursa Major,F


In [54]:
df = fit_coord(151500, 1925, n_HI = 2, HI_bounds = ([0, -15, 0], [20, 15, 30]), Cloud = 'Camelopardalis')
df.to_csv("fit_coord_saved_dfs/151500p1925_fitted.csv", index = False)
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),integral,integral error,Cloud,Upper Limit
1,151500,1925,HI,1,20.000000,1.010964,-0.746891,0.089165,1.846438,0.115768,150.858240,0.991445,Camelopardalis,F
2,151500,1925,HI,2,13.944734,0.757856,-1.122326,0.279188,8.748631,0.360983,352.311040,2.271686,Camelopardalis,F
3,151500,1925,67,1,0.022660,0.002330,-1.235308,0.154193,1.298628,0.154193,0.060443,0.004323,Camelopardalis,F
4,151500,1925,67,2,0.002335,0.757856,0.925913,0.279188,0.925913,0.360983,0.003032,0.000360,Camelopardalis,T
5,151500,1925,65,1,0.011849,0.002902,-1.449924,0.234809,0.812572,0.225325,0.023416,0.004654,Camelopardalis,F
6,151500,1925,65,2,0.002899,0.757856,-1.235308,0.279188,1.298628,0.360983,0.002355,0.000653,Camelopardalis,T
7,151500,1925,CO,1,1.513998,0.124205,-1.399332,0.066468,0.702123,0.066562,2.139098,0.151404,Camelopardalis,F
8,151500,1925,CO,2,0.134442,0.757856,-1.449924,0.279188,0.812572,0.360983,0.094395,0.008949,Camelopardalis,T


In [55]:
df = fit_coord(156540, 3513, n_67 = 2, n_HI = 2, n_65 = 2, OH67_p0 = (1, 0, 3), OH65_p0 = (1, 0, 3)
          ,OH67_bounds = ([0, -10, 0], [1, 2, 5]), OH65_bounds = ([0, -10, 0], [5, 2, 5]),
          CO_bounds = ([0, -10, 0], [2, 0, 5]), HI_bounds = ([0, -25, 0], [30, 10, 50]), Cloud = 'Ursa Major')
df.to_csv("fit_coord_saved_dfs/156540p3513_fitted.csv", index = False)
df
#weird CO behavior when bounds applied

[0.0023164  0.71068996 0.78609481 0.00306792 0.15336109 0.13834809]


,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),integral,integral error,Cloud,Upper Limit
1,156540,3513,HI,1,1.106760,0.105039,-21.970747,3.795141,35.380713,3.581406,209.858560,7.734672,Ursa Major,F
2,156540,3513,HI,2,14.720045,0.314741,-0.735552,0.079036,3.280452,0.084756,114.675200,2.401308,Ursa Major,F
3,156540,3513,67,1,0.017449,0.003448,0.426708,0.145289,0.666732,0.161628,0.025004,0.002970,Ursa Major,F
4,156540,3513,67,2,0.032019,0.002822,-3.067126,0.090899,0.883919,0.091281,0.063872,0.003638,Ursa Major,F
5,156540,3513,65,1,0.006454,0.002316,0.436334,0.710690,1.418376,0.786095,0.022087,0.005089,Ursa Major,F
6,156540,3513,65,2,0.022312,0.003068,-3.047328,0.153361,0.781596,0.138348,0.042941,0.003942,Ursa Major,F
7,156540,3513,CO,1,1.725987,0.114585,-3.363716,0.052656,0.687417,0.052742,2.480669,0.142233,Ursa Major,F
8,156540,3513,CO,2,0.126297,0.314741,0.436334,0.079036,1.418376,0.084756,0.086818,0.006661,Ursa Major,T


In [62]:
df = fit_coord(156640, 3250, Cloud = 'Ursa Major', OH67_bounds = ([0, -5, 0], [1, 5, 5]))
df.to_csv("fit_coord_saved_dfs/156640p3250_fitted.csv", index = False)
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),integral,integral error,Cloud,Upper Limit
1,156640,3250,HI,1,23.456820,0.679447,-0.742946,0.095651,2.861469,0.095685,134.853120,4.547945,Ursa Major,F
2,156640,3250,67,1,0.005350,4.088013,-0.657100,51.949659,0.303745,102.735789,0.002210,0.002247,Ursa Major,F
3,156640,3250,65,1,0.002425,0.001370,0.949288,2.455072,3.763955,2.455072,0.023622,0.006867,Ursa Major,F
4,156640,3250,CO,1,1.742431,0.101835,-0.815809,0.068763,1.018795,0.068763,3.762343,0.195941,Ursa Major,F
